In [1]:
# -*- coding: utf-8 -*-
"""
CELL 1: SYSTEM SETUP & INSTALLATION
- Environment detection (GPU/CPU)
- Package installation with progress tracking
- Directory structure creation
- System optimization for dual-stream processing
"""

import sys
import subprocess
import time
import os
import platform
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Check if running in Colab
try:
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False
    print("⚠️ Not running in Colab environment")

def setup_system():
    """Setup system environment and install required packages"""
    print("📦 CELL 1: SYSTEM SETUP & INSTALLATION")
    print("=" * 45)

    # Environment Detection
    print("🔍 ENVIRONMENT DETECTION:")
    print(f"├─ Platform: {platform.system()} {platform.release()}")
    print(f"├─ Python: {sys.version.split()[0]}")
    print(f"├─ Working Directory: {os.getcwd()}")
    print(f"└─ Colab Environment: {COLAB_ENV}")

    # GPU Detection
    gpu_available = False
    try:
        import torch
        gpu_available = torch.cuda.is_available()
        if gpu_available:
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"🚀 GPU: ✅ {gpu_name} ({gpu_memory:.1f}GB)")
            print("   └─ Will be used for HIGH-QUALITY face recognition")
        else:
            print(f"💻 GPU: ❌ CPU mode")
            print("   └─ CPU will handle both person detection and face recognition")
    except ImportError:
        print(f"📦 PyTorch: Installing...")

    # Install packages with progress tracking
    print(f"\n📦 INSTALLING PACKAGES:")

    packages = [
        'torch>=1.9.0',
        'torchvision>=0.10.0',
        'insightface>=0.7.3',
        'opencv-python>=4.5.0',
        'scikit-learn>=1.0.0',
        'matplotlib>=3.5.0',
        'pandas>=1.3.0',
        'tqdm>=4.62.0',
        'pillow>=8.3.0',
        'numpy>=1.21.0',
        'plotly>=5.0.0',
        'ipywidgets>=7.6.0' if COLAB_ENV else 'ipywidgets>=7.6.0',
        'psutil>=5.8.0'
    ]

    for i, package in enumerate(packages, 1):
        try:
            print(f"├─ [{i}/{len(packages)}] Installing {package.split('>=')[0]}...")
            result = subprocess.run(
                [sys.executable, '-m', 'pip', 'install', package, '--quiet'],
                capture_output=True, text=True, timeout=120
            )
            if result.returncode == 0:
                print(f"    ✅ Installed successfully")
            else:
                print(f"    ⚠️ Already installed or skipped")
        except subprocess.TimeoutExpired:
            print(f"    ⚠️ Installation timeout, continuing...")
        except Exception as e:
            print(f"    ❌ Error: {e}")

    # Create directory structure for Pipeline V1
    if COLAB_ENV:
        work_dir = Path('/content')
    else:
        work_dir = Path.cwd()

    # Pipeline V1 directory structure
    data_dir = work_dir / 'attendance_data'
    employees_dir = data_dir / 'employees'          # Employee photos
    videos_dir = data_dir / 'videos'                # Uploaded videos
    snapshots_dir = data_dir / 'snapshots'          # High-quality snapshots
    exports_dir = data_dir / 'exports'              # Reports and exports
    db_dir = data_dir / 'database'                  # SQLite database

    directories = [data_dir, employees_dir, videos_dir, snapshots_dir, exports_dir, db_dir]

    for directory in directories:
        directory.mkdir(parents=True, exist_ok=True)

    print(f"\n📁 PIPELINE V1 DIRECTORY STRUCTURE:")
    print(f"├─ Data Root: {data_dir}")
    print(f"├─ Employees: {employees_dir}")
    print(f"├─ Videos: {videos_dir}")
    print(f"├─ Snapshots: {snapshots_dir}")
    print(f"├─ Exports: {exports_dir}")
    print(f"└─ Database: {db_dir}")

    print(f"\n🎉 SYSTEM SETUP COMPLETED!")
    print(f"Ready for Pipeline V1 dual-stream processing!")
    print("=" * 45)

    return {
        'gpu_available': gpu_available,
        'work_dir': work_dir,
        'data_dir': data_dir,
        'employees_dir': employees_dir,
        'videos_dir': videos_dir,
        'snapshots_dir': snapshots_dir,
        'exports_dir': exports_dir,
        'db_dir': db_dir
    }

# Run setup when imported
if __name__ == "__main__":
    config = setup_system()
    print("✅ Cell 1 completed. Import this file to use setup_system() function.")

📦 CELL 1: SYSTEM SETUP & INSTALLATION
🔍 ENVIRONMENT DETECTION:
├─ Platform: Linux 6.1.123+
├─ Python: 3.11.13
├─ Working Directory: /content
└─ Colab Environment: True
🚀 GPU: ✅ Tesla T4 (15.8GB)
   └─ Will be used for HIGH-QUALITY face recognition

📦 INSTALLING PACKAGES:
├─ [1/13] Installing torch...
    ✅ Installed successfully
├─ [2/13] Installing torchvision...
    ✅ Installed successfully
├─ [3/13] Installing insightface...
    ✅ Installed successfully
├─ [4/13] Installing opencv-python...
    ✅ Installed successfully
├─ [5/13] Installing scikit-learn...
    ✅ Installed successfully
├─ [6/13] Installing matplotlib...
    ✅ Installed successfully
├─ [7/13] Installing pandas...
    ✅ Installed successfully
├─ [8/13] Installing tqdm...
    ✅ Installed successfully
├─ [9/13] Installing pillow...
    ✅ Installed successfully
├─ [10/13] Installing numpy...
    ✅ Installed successfully
├─ [11/13] Installing plotly...
    ✅ Installed successfully
├─ [12/13] Installing ipywidgets...
    ✅ I

In [1]:
!pip install onnxruntime-gpu onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 60.6 MB/s eta 0:00:00


In [2]:
# -*- coding: utf-8 -*-
"""
CELL 2: DUAL-STREAM AI SYSTEM
- Initialize SCRFD + ArcFace models theo Pipeline V1
- Setup dual-stream processing:
  + CPU stream for person detection (low quality)
  + GPU stream for face recognition (high quality snapshots)
- Performance testing and optimization
"""

import cv2
import numpy as np
import time
import logging
from typing import List, Dict, Optional, Tuple, Any

# AI and ML imports
import insightface
from insightface.app import FaceAnalysis

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class DualStreamAISystem:
    """
    Dual-Stream AI System theo Pipeline V1
    - CPU Stream: Person detection on low-quality frames
    - GPU Stream: Face recognition on high-quality snapshots
    """

    def __init__(self, model_pack='buffalo_l'):
        self.model_pack = model_pack
        self.gpu_available = False

        # Dual stream models
        self.cpu_app = None      # CPU for person detection
        self.gpu_app = None      # GPU for face recognition

        # Performance stats
        self.performance_stats = {
            'total_inferences': 0,
            'cpu_detections': 0,
            'gpu_recognitions': 0,
            'avg_cpu_latency_ms': 0.0,
            'avg_gpu_latency_ms': 0.0,
            'total_cpu_time': 0.0,
            'total_gpu_time': 0.0,
            'gpu_available': False
        }

        print("🤖 CELL 2: DUAL-STREAM AI MODELS INITIALIZATION")
        print("=" * 50)
        self._init_dual_stream_models()
        self._test_performance()

    def _init_dual_stream_models(self):
        """Initialize dual-stream models according to Pipeline V1"""
        try:
            import torch
            self.gpu_available = torch.cuda.is_available()
            self.performance_stats['gpu_available'] = self.gpu_available

            if self.gpu_available:
                print(f"🚀 GPU Mode: {torch.cuda.get_device_name(0)}")
                print("├─ CPU Stream: Person detection (low quality)")
                print("└─ GPU Stream: Face recognition (high quality)")

                # Setup providers for dual stream
                cpu_providers = ['CPUExecutionProvider']
                gpu_providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']

                # Initialize CPU stream for person detection
                print(f"\n📦 Loading CPU Stream - {self.model_pack}...")
                self.cpu_app = FaceAnalysis(name=self.model_pack, providers=cpu_providers)
                self.cpu_app.prepare(ctx_id=-1, det_size=(320, 320), det_thresh=0.3)  # Lower quality for CPU
                print("✅ CPU Stream initialized (Person Detection)")

                # Initialize GPU stream for face recognition
                print(f"\n📦 Loading GPU Stream - {self.model_pack}...")
                self.gpu_app = FaceAnalysis(name=self.model_pack, providers=gpu_providers)
                self.gpu_app.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)   # Higher quality for GPU
                print("✅ GPU Stream initialized (Face Recognition)")

            else:
                print("💻 CPU Only Mode: Single stream processing")
                print("└─ CPU will handle both detection and recognition")

                # Single CPU stream
                cpu_providers = ['CPUExecutionProvider']
                self.cpu_app = FaceAnalysis(name=self.model_pack, providers=cpu_providers)
                self.cpu_app.prepare(ctx_id=-1, det_size=(640, 640), det_thresh=0.5)
                print("✅ CPU Stream initialized (Detection + Recognition)")

        except Exception as e:
            logger.error(f"Model loading failed: {e}")
            raise

    def detect_person_cpu(self, low_quality_frame: np.ndarray) -> List[Dict]:
        """
        CPU Stream: Person detection on low-quality frames
        Used for initial person detection in zones (Pipeline V1)
        """
        start_time = time.time()

        try:
            # CPU detection with lower threshold for person detection
            faces = self.cpu_app.get(low_quality_frame)

            # Update CPU performance stats
            cpu_time = (time.time() - start_time) * 1000
            self.performance_stats['cpu_detections'] += 1
            self.performance_stats['total_cpu_time'] += cpu_time
            if self.performance_stats['cpu_detections'] > 0:
                self.performance_stats['avg_cpu_latency_ms'] = (
                    self.performance_stats['total_cpu_time'] /
                    self.performance_stats['cpu_detections']
                )

            # Format results for person detection
            results = []
            for face in faces:
                result = {
                    'bbox': face.bbox,
                    'det_score': float(face.det_score),
                    'landmarks': getattr(face, 'kps', None),
                    'stream_type': 'cpu_detection',
                    'processing_time_ms': cpu_time
                }
                results.append(result)

            return results

        except Exception as e:
            logger.error(f"CPU detection error: {e}")
            return []

    def recognize_face_gpu(self, high_quality_snapshot: np.ndarray) -> List[Dict]:
        """
        GPU Stream: Face recognition on high-quality snapshots
        Used for actual employee recognition (Pipeline V1)
        """
        start_time = time.time()

        try:
            # Use GPU stream if available, otherwise fallback to CPU
            app = self.gpu_app if self.gpu_available else self.cpu_app
            faces = app.get(high_quality_snapshot)

            # Update GPU performance stats
            gpu_time = (time.time() - start_time) * 1000
            self.performance_stats['gpu_recognitions'] += 1
            self.performance_stats['total_gpu_time'] += gpu_time
            if self.performance_stats['gpu_recognitions'] > 0:
                self.performance_stats['avg_gpu_latency_ms'] = (
                    self.performance_stats['total_gpu_time'] /
                    self.performance_stats['gpu_recognitions']
                )

            # Format results with embeddings for recognition
            results = []
            for face in faces:
                result = {
                    'bbox': face.bbox,
                    'det_score': float(face.det_score),
                    'landmarks': getattr(face, 'kps', None),
                    'embedding': face.embedding,  # 512-dim ArcFace embedding
                    'age': getattr(face, 'age', None),
                    'gender': getattr(face, 'gender', None),
                    'stream_type': 'gpu_recognition',
                    'processing_time_ms': gpu_time,
                    'embedding_norm': float(np.linalg.norm(face.embedding))
                }
                results.append(result)

            return results

        except Exception as e:
            logger.error(f"GPU recognition error: {e}")
            return []

    def process_dual_stream(self, low_quality_frame: np.ndarray,
                           high_quality_frame: np.ndarray = None) -> Dict:
        """
        Process dual stream according to Pipeline V1:
        1. CPU detects person in low-quality frame
        2. If person detected, GPU processes high-quality frame for recognition
        """
        result = {
            'person_detected': False,
            'faces_recognized': [],
            'cpu_detections': [],
            'processing_summary': {
                'cpu_time_ms': 0.0,
                'gpu_time_ms': 0.0,
                'total_time_ms': 0.0
            }
        }

        total_start = time.time()

        # Step 1: CPU person detection on low-quality frame
        cpu_detections = self.detect_person_cpu(low_quality_frame)
        result['cpu_detections'] = cpu_detections

        if cpu_detections:
            result['person_detected'] = True

            # Step 2: GPU face recognition on high-quality frame (if provided)
            if high_quality_frame is not None:
                gpu_recognitions = self.recognize_face_gpu(high_quality_frame)
                result['faces_recognized'] = gpu_recognitions
            else:
                # Fallback: use low-quality frame for recognition
                gpu_recognitions = self.recognize_face_gpu(low_quality_frame)
                result['faces_recognized'] = gpu_recognitions

        # Update performance summary
        total_time = (time.time() - total_start) * 1000
        result['processing_summary']['total_time_ms'] = total_time
        result['processing_summary']['cpu_time_ms'] = sum(d.get('processing_time_ms', 0) for d in cpu_detections)
        result['processing_summary']['gpu_time_ms'] = sum(f.get('processing_time_ms', 0) for f in result['faces_recognized'])

        self.performance_stats['total_inferences'] += 1

        return result

    def _test_performance(self):
        """Test dual-stream performance"""
        print("\n🧪 DUAL-STREAM PERFORMANCE TEST:")
        print("-" * 35)

        try:
            # Create test images
            low_quality_test = np.random.randint(0, 255, (240, 320, 3), dtype=np.uint8)   # Sub-stream
            high_quality_test = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)  # Main-stream

            # Warm up
            for _ in range(3):
                self.process_dual_stream(low_quality_test, high_quality_test)

            # Performance test
            times = []
            for i in range(5):
                start_time = time.time()
                result = self.process_dual_stream(low_quality_test, high_quality_test)
                end_time = time.time()
                times.append((end_time - start_time) * 1000)

            avg_latency = np.mean(times)
            min_latency = np.min(times)

            print(f"├─ Average Total Latency: {avg_latency:.1f}ms")
            print(f"├─ Best Total Latency: {min_latency:.1f}ms")
            print(f"├─ CPU Detection Avg: {self.performance_stats['avg_cpu_latency_ms']:.1f}ms")
            print(f"├─ GPU Recognition Avg: {self.performance_stats['avg_gpu_latency_ms']:.1f}ms")
            print(f"├─ GPU Available: {self.gpu_available}")
            print(f"└─ Model Pack: {self.model_pack}")

            # Performance rating
            if avg_latency < 100:
                print("🌟 Performance: EXCELLENT (real-time capable)")
            elif avg_latency < 300:
                print("👍 Performance: GOOD")
            else:
                print("⚠️ Performance: Needs optimization")

        except Exception as e:
            logger.warning(f"⚠️ Performance test error: {e}")

    def get_performance_stats(self) -> Dict:
        """Get comprehensive performance statistics"""
        stats = self.performance_stats.copy()

        # Add system information
        if self.gpu_available:
            import torch
            stats['gpu_name'] = torch.cuda.get_device_name(0)
            stats['gpu_memory_allocated_mb'] = torch.cuda.memory_allocated() / 1024 / 1024

        stats['model_pack'] = self.model_pack
        stats['dual_stream_enabled'] = self.gpu_available

        return stats

    def cleanup(self):
        """Cleanup resources"""
        try:
            if hasattr(self, 'cpu_app'):
                del self.cpu_app
            if hasattr(self, 'gpu_app'):
                del self.gpu_app

            # Clear GPU cache if available
            if self.gpu_available:
                import torch
                torch.cuda.empty_cache()

            logger.info("✅ AI Models cleanup completed")

        except Exception as e:
            logger.warning(f"⚠️ Cleanup warning: {e}")

# Initialize AI system when imported
if __name__ == "__main__":
    ai_system = DualStreamAISystem()
    print("✅ Cell 2 completed. AI System ready for dual-stream processing.")

🤖 CELL 2: DUAL-STREAM AI MODELS INITIALIZATION
🚀 GPU Mode: Tesla T4
├─ CPU Stream: Person detection (low quality)
└─ GPU Stream: Face recognition (high quality)

📦 Loading CPU Stream - buffalo_l...
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionPro

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ GPU Stream

In [3]:
# -*- coding: utf-8 -*-
"""
CELL 3: SQLITE DATABASE WITH VECTOR SIMILARITY SEARCH
- SQLite database design theo Pipeline V1
- Vector similarity search using cosine distance
- Employee registration with face embeddings
- Attendance logging with comprehensive metadata
- Compatible với PostgreSQL migration path
"""

import sqlite3
import json
import numpy as np
import logging
import time
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple, Any
from sklearn.metrics.pairwise import cosine_similarity
import os
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class AttendanceDatabaseSQLite:
    """
    SQLite database with vector operations simulation
    Compatible with PostgreSQL + pgvector migration (Pipeline V1)
    """

    def __init__(self, db_path: str = "attendance_pipeline_v1.db", enable_wal: bool = True):
        """Initialize SQLite database with Pipeline V1 schema"""
        print("\n🗄️ CELL 3: SQLITE DATABASE INITIALIZATION")
        print("=" * 45)

        self.db_path = db_path if isinstance(db_path, Path) else Path(db_path)
        self.enable_wal = enable_wal

        print(f"📊 Database: {self.db_path}")
        print("├─ Vector Similarity: Cosine distance simulation")
        print("├─ Employee Management: Face embeddings storage")
        print("├─ Attendance Logging: Comprehensive metadata")
        print("└─ Migration Ready: PostgreSQL + pgvector compatible")

        # Create database directory if needed
        self.db_path.parent.mkdir(parents=True, exist_ok=True)

        # Initialize connection
        self._init_connection()

        # Setup schema
        self._create_tables()
        self._create_indexes()
        self._setup_performance_optimization()
        self._initialize_system_config()

        print("✅ Database initialization completed")

    def _init_connection(self):
        """Initialize database connection with optimizations"""
        self.conn = sqlite3.connect(
            str(self.db_path),
            check_same_thread=False,
            timeout=30.0
        )

        # Enable row factory for dict-like access
        self.conn.row_factory = sqlite3.Row

        # Enable WAL mode for better concurrency
        if self.enable_wal:
            self.conn.execute("PRAGMA journal_mode=WAL")

        # Performance optimizations
        self.conn.execute("PRAGMA synchronous=NORMAL")
        self.conn.execute("PRAGMA cache_size=10000")
        self.conn.execute("PRAGMA temp_store=MEMORY")
        self.conn.execute("PRAGMA mmap_size=268435456")  # 256MB

        print("🔧 Database connection optimized")

    def _create_tables(self):
        """Create Pipeline V1 compatible schema"""
        cursor = self.conn.cursor()

        # Employees table with face embeddings (JSON format for SQLite)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_code TEXT UNIQUE NOT NULL,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            department TEXT,
            position TEXT,
            face_embeddings TEXT,  -- JSON array of 512-dim vectors
            embedding_count INTEGER DEFAULT 0,
            registration_quality REAL DEFAULT 0.0,
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # Attendance logs with dual-stream metadata
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS attendance_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_id INTEGER,
            event_type TEXT NOT NULL CHECK (event_type IN ('check_in', 'check_out', 'video_detection')),
            timestamp TIMESTAMP NOT NULL,
            confidence REAL NOT NULL,

            -- Dual-stream processing metadata
            detection_stream TEXT,  -- 'cpu' or 'gpu'
            recognition_stream TEXT,  -- 'cpu' or 'gpu'
            cpu_processing_time_ms REAL,
            gpu_processing_time_ms REAL,

            -- Video processing metadata
            video_source TEXT,
            frame_number INTEGER,
            snapshot_path TEXT,

            -- Face detection metadata
            face_bbox TEXT,  -- JSON [x1, y1, x2, y2]
            face_quality REAL,
            face_landmarks TEXT,  -- JSON landmarks

            -- Business logic metadata
            zone_detected TEXT,
            cooldown_applied BOOLEAN DEFAULT 0,
            work_hours_valid BOOLEAN DEFAULT 1,

            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (employee_id) REFERENCES employees (id)
        )
        """)

        # Individual face registrations for quality tracking
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS face_registrations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_id INTEGER,
            image_path TEXT NOT NULL,
            embedding TEXT NOT NULL,  -- JSON 512-dim vector
            quality_score REAL,
            detection_confidence REAL,
            landmarks TEXT,  -- JSON landmarks
            image_metadata TEXT,  -- JSON metadata (size, format, etc.)
            stream_type TEXT,  -- 'cpu' or 'gpu'
            processing_time_ms REAL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (employee_id) REFERENCES employees (id)
        )
        """)

        # System configuration for Pipeline V1 business rules
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS system_config (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            config_key TEXT UNIQUE NOT NULL,
            config_value TEXT NOT NULL,
            description TEXT,
            config_type TEXT DEFAULT 'string',
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # Performance metrics for dual-stream monitoring
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS performance_metrics (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            metric_type TEXT NOT NULL,
            metric_value REAL NOT NULL,
            stream_type TEXT,  -- 'cpu', 'gpu', 'dual'
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            metadata TEXT  -- JSON additional context
        )
        """)

        self.conn.commit()
        print("🗄️ Pipeline V1 tables created")

    def _create_indexes(self):
        """Create indexes for performance"""
        cursor = self.conn.cursor()

        indexes = [
            # Attendance performance indexes
            "CREATE INDEX IF NOT EXISTS idx_attendance_employee_date ON attendance_logs(employee_id, DATE(timestamp))",
            "CREATE INDEX IF NOT EXISTS idx_attendance_timestamp ON attendance_logs(timestamp)",
            "CREATE INDEX IF NOT EXISTS idx_attendance_event_type ON attendance_logs(event_type)",

            # Employee indexes
            "CREATE INDEX IF NOT EXISTS idx_employees_active ON employees(is_active)",
            "CREATE INDEX IF NOT EXISTS idx_employees_code ON employees(employee_code)",

            # Face registration indexes
            "CREATE INDEX IF NOT EXISTS idx_face_registrations_employee ON face_registrations(employee_id)",
            "CREATE INDEX IF NOT EXISTS idx_face_registrations_quality ON face_registrations(quality_score)",

            # System config indexes
            "CREATE INDEX IF NOT EXISTS idx_system_config_key ON system_config(config_key)",

            # Performance metrics indexes
            "CREATE INDEX IF NOT EXISTS idx_performance_metrics_type ON performance_metrics(metric_type, timestamp)",
            "CREATE INDEX IF NOT EXISTS idx_performance_metrics_stream ON performance_metrics(stream_type, timestamp)"
        ]

        for index_sql in indexes:
            cursor.execute(index_sql)

        self.conn.commit()
        print("📇 Database indexes created")

    def _setup_performance_optimization(self):
        """Setup additional performance optimizations"""
        cursor = self.conn.cursor()

        # Analyze tables for query optimization
        cursor.execute("ANALYZE")

        # Enable automatic index recommendations
        cursor.execute("PRAGMA optimize")

        self.conn.commit()
        print("⚡ Performance optimization applied")

    def _initialize_system_config(self):
        """Initialize Pipeline V1 default configuration"""
        default_configs = [
            # Recognition thresholds
            ("recognition_threshold", "0.65", "Minimum similarity threshold for face recognition", "float"),
            ("detection_threshold_cpu", "0.3", "CPU detection confidence threshold", "float"),
            ("detection_threshold_gpu", "0.5", "GPU detection confidence threshold", "float"),

            # Business logic
            ("cooldown_minutes", "30", "Minimum minutes between attendance records", "int"),
            ("work_hours_start", "07:00", "Work day start time", "time"),
            ("work_hours_end", "19:00", "Work day end time", "time"),

            # Dual-stream processing
            ("enable_dual_stream", "true", "Enable dual-stream processing", "bool"),
            ("cpu_frame_resolution", "320x240", "CPU processing frame resolution", "string"),
            ("gpu_frame_resolution", "640x480", "GPU processing frame resolution", "string"),

            # Quality settings
            ("face_quality_threshold", "0.3", "Minimum face quality score", "float"),
            ("max_face_registrations", "10", "Maximum face images per employee", "int"),

            # Performance settings
            ("max_processing_fps", "5", "Maximum video processing FPS", "int"),
            ("snapshot_quality", "95", "JPEG quality for snapshots", "int"),

            # Data retention
            ("backup_retention_days", "30", "Days to retain backup files", "int"),
            ("performance_metrics_retention_days", "7", "Days to retain performance metrics", "int")
        ]

        cursor = self.conn.cursor()
        for key, value, description, config_type in default_configs:
            cursor.execute("""
            INSERT OR IGNORE INTO system_config (config_key, config_value, description, config_type)
            VALUES (?, ?, ?, ?)
            """, (key, value, description, config_type))

        self.conn.commit()
        print("⚙️ Pipeline V1 configuration initialized")

    def register_employee(self, employee_data: Dict, face_embeddings: List[np.ndarray],
                         registration_metadata: List[Dict] = None) -> Optional[int]:
        """
        Employee registration with multiple face embeddings
        Compatible with Pipeline V1 dual-stream processing
        """
        cursor = self.conn.cursor()

        try:
            print(f"📝 Registering employee: {employee_data['name']}")

            # Begin transaction
            cursor.execute("BEGIN TRANSACTION")

            # Insert employee record
            cursor.execute("""
            INSERT INTO employees (employee_code, name, email, department, position, embedding_count)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (
                employee_data['employee_code'],
                employee_data['name'],
                employee_data['email'],
                employee_data.get('department', ''),
                employee_data.get('position', ''),
                len(face_embeddings)
            ))

            employee_id = cursor.lastrowid

            # Store individual face registrations with metadata
            registration_qualities = []

            for i, embedding in enumerate(face_embeddings):
                # Get metadata for this registration
                metadata = registration_metadata[i] if registration_metadata and i < len(registration_metadata) else {}

                # Calculate quality score
                quality_score = self._calculate_embedding_quality(embedding, metadata)
                registration_qualities.append(quality_score)

                cursor.execute("""
                INSERT INTO face_registrations
                (employee_id, image_path, embedding, quality_score, detection_confidence,
                 landmarks, image_metadata, stream_type, processing_time_ms)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    employee_id,
                    f"registration_{employee_id}_{i}.jpg",
                    json.dumps(embedding.tolist()),
                    quality_score,
                    metadata.get('detection_confidence', 0.0),
                    json.dumps(metadata.get('landmarks', []).tolist() if hasattr(metadata.get('landmarks', []), 'tolist') else metadata.get('landmarks', [])),
                    json.dumps(metadata.get('image_metadata', {})),
                    metadata.get('stream_type', 'gpu'),
                    metadata.get('processing_time_ms', 0.0)
                ))

            # Calculate average embedding and overall quality
            if face_embeddings:
                avg_embedding = np.mean(face_embeddings, axis=0)
                avg_quality = np.mean(registration_qualities)

                # Update employee with averaged embedding
                cursor.execute("""
                UPDATE employees
                SET face_embeddings = ?, registration_quality = ?, updated_at = CURRENT_TIMESTAMP
                WHERE id = ?
                """, (json.dumps(avg_embedding.tolist()), avg_quality, employee_id))

            # Commit transaction
            cursor.execute("COMMIT")

            print(f"✅ Employee registered successfully: ID {employee_id}")
            print(f"   ├─ Face embeddings: {len(face_embeddings)}")
            print(f"   └─ Average quality: {avg_quality:.3f}")

            # Record performance metric
            self._record_performance_metric("employee_registration", 1.0, "dual", {"employee_id": employee_id})

            return employee_id

        except Exception as e:
            cursor.execute("ROLLBACK")
            logger.error(f"❌ Employee registration failed: {e}")
            return None

    def find_employee_by_embedding(self, embedding: np.ndarray, threshold: float = 0.65) -> Optional[Dict]:
        """
        Vector similarity search using cosine distance
        Optimized for Pipeline V1 performance requirements
        """
        start_time = time.time()

        try:
            cursor = self.conn.cursor()

            # Get all active employees with embeddings
            cursor.execute("""
            SELECT id, employee_code, name, email, department, face_embeddings, registration_quality
            FROM employees
            WHERE is_active = 1 AND face_embeddings IS NOT NULL
            """)

            employees = cursor.fetchall()

            if not employees:
                return None

            best_match = None
            best_similarity = 0.0

            # Calculate similarities with vectorized operations
            query_embedding = embedding.reshape(1, -1)

            for emp in employees:
                try:
                    stored_embedding = np.array(json.loads(emp['face_embeddings']))
                    stored_embedding = stored_embedding.reshape(1, -1)

                    # Calculate cosine similarity
                    similarity = cosine_similarity(query_embedding, stored_embedding)[0][0]

                    if similarity > best_similarity and similarity > threshold:
                        best_similarity = similarity
                        best_match = {
                            'id': emp['id'],
                            'employee_code': emp['employee_code'],
                            'name': emp['name'],
                            'email': emp['email'],
                            'department': emp['department'],
                            'similarity': similarity,
                            'registration_quality': emp['registration_quality']
                        }

                except Exception as e:
                    logger.warning(f"Error processing employee {emp['id']}: {e}")
                    continue

            # Record performance metric
            search_time = (time.time() - start_time) * 1000
            self._record_performance_metric("embedding_search_ms", search_time, "gpu", {
                "employees_searched": len(employees),
                "match_found": best_match is not None,
                "threshold": threshold
            })

            return best_match

        except Exception as e:
            logger.error(f"❌ Employee search error: {e}")
            return None

    def record_attendance(self, employee_id: int, event_type: str, confidence: float,
                         timestamp: str = None, dual_stream_metadata: Dict = None, **kwargs) -> Optional[int]:
        """
        Record attendance with comprehensive dual-stream metadata
        Pipeline V1 compatible with business logic
        """
        if timestamp is None:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        try:
            cursor = self.conn.cursor()

            # Extract dual-stream metadata
            metadata = dual_stream_metadata or {}

            # Prepare attendance record
            cursor.execute("""
            INSERT INTO attendance_logs
            (employee_id, event_type, timestamp, confidence,
             detection_stream, recognition_stream, cpu_processing_time_ms, gpu_processing_time_ms,
             video_source, frame_number, snapshot_path,
             face_bbox, face_quality, face_landmarks,
             zone_detected, cooldown_applied, work_hours_valid)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                employee_id,
                event_type,
                timestamp,
                confidence,
                metadata.get('detection_stream', 'gpu'),
                metadata.get('recognition_stream', 'gpu'),
                metadata.get('cpu_processing_time_ms', 0.0),
                metadata.get('gpu_processing_time_ms', 0.0),
                kwargs.get('video_source', ''),
                kwargs.get('frame_number', 0),
                kwargs.get('snapshot_path', ''),
                json.dumps(kwargs.get('face_bbox', []).tolist() if hasattr(kwargs.get('face_bbox', []), 'tolist') else kwargs.get('face_bbox', [])),
                kwargs.get('face_quality', 0.0),
                json.dumps(kwargs.get('face_landmarks', []).tolist() if hasattr(kwargs.get('face_landmarks', []), 'tolist') else kwargs.get('face_landmarks', [])),
                kwargs.get('zone_detected', 'main_entrance'),
                kwargs.get('cooldown_applied', False),
                kwargs.get('work_hours_valid', True)
            ))

            attendance_id = cursor.lastrowid
            self.conn.commit()

            logger.info(f"📋 Attendance recorded: Employee {employee_id}, {event_type}, ID {attendance_id}")

            # Record performance metric
            self._record_performance_metric("attendance_recorded", 1.0, "dual", {
                "employee_id": employee_id,
                "event_type": event_type,
                "confidence": confidence
            })

            return attendance_id

        except Exception as e:
            logger.error(f"❌ Attendance recording error: {e}")
            return None

    def _calculate_embedding_quality(self, embedding: np.ndarray, metadata: Dict = None) -> float:
        """Calculate quality score for face embedding"""
        try:
            # Base quality metrics
            norm = np.linalg.norm(embedding)
            variance = np.var(embedding)

            # Normalize metrics
            norm_score = min(norm / 1.0, 1.0)  # Good embeddings have norm around 1
            variance_score = min(variance * 10, 1.0)  # Higher variance usually better

            base_quality = (norm_score + variance_score) / 2

            # Factor in detection confidence if available
            if metadata and 'detection_confidence' in metadata:
                det_confidence = metadata['detection_confidence']
                quality = (base_quality + det_confidence) / 2
            else:
                quality = base_quality

            return min(max(quality, 0.0), 1.0)

        except:
            return 0.5  # Default quality

    def _record_performance_metric(self, metric_type: str, metric_value: float,
                                  stream_type: str = None, metadata: Dict = None):
        """Record performance metric for monitoring"""
        try:
            cursor = self.conn.cursor()
            cursor.execute("""
            INSERT INTO performance_metrics (metric_type, metric_value, stream_type, metadata)
            VALUES (?, ?, ?, ?)
            """, (metric_type, metric_value, stream_type, json.dumps(metadata) if metadata else None))
            self.conn.commit()
        except Exception as e:
            logger.warning(f"Failed to record performance metric: {e}")

    def get_statistics(self) -> Dict:
        """Get comprehensive database statistics"""
        cursor = self.conn.cursor()

        stats = {}

        # Basic counts
        cursor.execute("SELECT COUNT(*) FROM employees WHERE is_active = 1")
        stats['total_active_employees'] = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM employees WHERE is_active = 1 AND face_embeddings IS NOT NULL")
        stats['employees_with_faces'] = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM attendance_logs")
        stats['total_attendance_logs'] = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM face_registrations")
        stats['total_face_registrations'] = cursor.fetchone()[0]

        # Today's statistics
        cursor.execute("SELECT COUNT(*) FROM attendance_logs WHERE DATE(timestamp) = DATE('now')")
        stats['todays_attendance_count'] = cursor.fetchone()[0]

        # Average quality scores
        cursor.execute("SELECT AVG(registration_quality) FROM employees WHERE registration_quality > 0")
        avg_quality = cursor.fetchone()[0]
        stats['avg_registration_quality'] = avg_quality if avg_quality else 0.0

        # Dual-stream performance
        cursor.execute("SELECT COUNT(*) FROM attendance_logs WHERE detection_stream = 'cpu'")
        stats['cpu_detections'] = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM attendance_logs WHERE recognition_stream = 'gpu'")
        stats['gpu_recognitions'] = cursor.fetchone()[0]

        # Recent activity
        cursor.execute("""
        SELECT COUNT(*) FROM attendance_logs
        WHERE timestamp >= datetime('now', '-24 hours')
        """)
        stats['activity_last_24h'] = cursor.fetchone()[0]

        return stats

    def get_attendance_logs(self, limit: int = 100) -> List[Dict]:
        """Get recent attendance logs with employee information"""
        cursor = self.conn.cursor()
        cursor.execute("""
        SELECT al.*, e.name, e.employee_code, e.department
        FROM attendance_logs al
        JOIN employees e ON al.employee_id = e.id
        ORDER BY al.timestamp DESC
        LIMIT ?
        """, (limit,))

        return [dict(row) for row in cursor.fetchall()]

    def close(self):
        """Close database connection"""
        try:
            if hasattr(self, 'conn'):
                self.conn.close()
            logger.info("✅ Database connection closed")
        except Exception as e:
            logger.warning(f"⚠️ Database close warning: {e}")

# Initialize database when imported
if __name__ == "__main__":
    database = AttendanceDatabaseSQLite()
    print("✅ Cell 3 completed. Database ready for Pipeline V1 operations.")


🗄️ CELL 3: SQLITE DATABASE INITIALIZATION
📊 Database: attendance_pipeline_v1.db
├─ Vector Similarity: Cosine distance simulation
├─ Employee Management: Face embeddings storage
├─ Attendance Logging: Comprehensive metadata
└─ Migration Ready: PostgreSQL + pgvector compatible
🔧 Database connection optimized
🗄️ Pipeline V1 tables created
📇 Database indexes created
⚡ Performance optimization applied
⚙️ Pipeline V1 configuration initialized
✅ Database initialization completed
✅ Cell 3 completed. Database ready for Pipeline V1 operations.


In [4]:
# -*- coding: utf-8 -*-
"""
CELL 4: EMPLOYEE MANAGEMENT SYSTEM
- Photo upload and validation
- Face detection and embedding extraction
- Quality assessment and filtering
- Batch registration with progress tracking
- Integration with dual-stream AI system
"""

import numpy as np
import cv2
import zipfile
import shutil
from pathlib import Path
import logging
from typing import List, Dict, Optional
from tqdm import tqdm
import pandas as pd

# Check if running in Colab
try:
    from google.colab import files
    from IPython.display import display, HTML
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class EmployeeManager:
    """
    Employee Management System for Pipeline V1
    Handles photo upload, face detection, and registration
    """

    def __init__(self, ai_system, database, config: Dict):
        self.ai_system = ai_system
        self.db = database
        self.config = config
        self.registered_employees = []

        print("\n👥 CELL 4: EMPLOYEE MANAGEMENT SYSTEM")
        print("=" * 45)
        print("├─ Photo Upload: ZIP files or individual images")
        print("├─ Face Detection: Dual-stream processing")
        print("├─ Quality Assessment: Multi-factor validation")
        print("└─ Batch Registration: Progress tracking")

    def upload_employee_photos(self):
        """Upload employee photos via Colab file picker"""
        if not COLAB_ENV:
            print("❌ This function requires Google Colab environment")
            return

        print("📤 EMPLOYEE PHOTO UPLOAD")
        print("=" * 30)
        print("📋 Supported formats:")
        print("├─ ZIP archives: employee_name/photo1.jpg, photo2.jpg, ...")
        print("├─ Individual images: employee_name.jpg")
        print("└─ Image formats: JPG, JPEG, PNG")

        try:
            uploaded = files.upload()

            for filename, content in uploaded.items():
                print(f"\n📁 Processing: {filename}")

                if filename.endswith('.zip'):
                    self._process_zip_file(filename, content)
                elif filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self._process_single_image(filename, content)
                else:
                    print(f"  ⚠️ Unsupported format: {filename}")

        except Exception as e:
            logger.error(f"Upload error: {e}")

    def _process_zip_file(self, filename: str, content: bytes):
        """Process ZIP file with employee folders"""
        zip_path = self.config['employees_dir'] / filename

        # Save ZIP file
        with open(zip_path, 'wb') as f:
            f.write(content)

        # Extract ZIP
        extract_dir = self.config['employees_dir'] / 'extracted'
        extract_dir.mkdir(exist_ok=True)

        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)

            print(f"  ✅ Extracted to: {extract_dir}")

            # Process extracted folders
            self._scan_employee_folders(extract_dir)

        except Exception as e:
            print(f"  ❌ ZIP extraction error: {e}")
        finally:
            # Cleanup
            if zip_path.exists():
                zip_path.unlink()

    def _process_single_image(self, filename: str, content: bytes):
        """Process single image file"""
        # Extract employee name from filename
        employee_name = filename.split('.')[0].replace('_', ' ').title()

        # Save image
        img_path = self.config['employees_dir'] / filename
        with open(img_path, 'wb') as f:
            f.write(content)

        # Process image
        try:
            image = cv2.imread(str(img_path))
            if image is not None:
                self._register_single_employee(employee_name, [image], [str(img_path)])
        except Exception as e:
            print(f"  ❌ Error processing {filename}: {e}")

    def _scan_employee_folders(self, base_dir: Path):
        """Scan and process employee folders"""
        employee_folders = [f for f in base_dir.iterdir()
                           if f.is_dir() and not f.name.startswith('.')]

        if not employee_folders:
            print(f"  ⚠️ No employee folders found")
            return

        print(f"  📁 Found {len(employee_folders)} employee folders")

        for folder in tqdm(employee_folders, desc="Processing employees"):
            employee_name = folder.name.replace('_', ' ').title()

            # Find image files
            image_files = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                image_files.extend(folder.glob(ext))

            if not image_files:
                print(f"  ⚠️ No images in {folder.name}")
                continue

            # Load images
            images = []
            image_paths = []
            for img_file in image_files:
                try:
                    img = cv2.imread(str(img_file))
                    if img is not None:
                        images.append(img)
                        image_paths.append(str(img_file))
                except:
                    continue

            if images:
                self._register_single_employee(employee_name, images, image_paths)

    def _register_single_employee(self, employee_name: str, images: List[np.ndarray], image_paths: List[str]):
        """Register single employee with dual-stream processing"""
        print(f"\n👤 Processing: {employee_name}")
        print("-" * (len(employee_name) + 15))

        # Extract face embeddings using dual-stream
        face_embeddings = []
        registration_metadata = []
        processed_count = 0

        for i, (image, img_path) in enumerate(zip(images, image_paths)):
            try:
                print(f"  📸 Image {i+1}/{len(images)}: {Path(img_path).name}")

                # Create low and high quality versions for dual-stream
                height, width = image.shape[:2]

                # Low quality for CPU detection
                low_quality = cv2.resize(image, (320, 240))

                # High quality for GPU recognition
                if width > 640 or height > 480:
                    # Resize maintaining aspect ratio
                    scale = min(640/width, 480/height)
                    new_width = int(width * scale)
                    new_height = int(height * scale)
                    high_quality = cv2.resize(image, (new_width, new_height))
                else:
                    high_quality = image.copy()

                # Process with dual-stream
                result = self.ai_system.process_dual_stream(low_quality, high_quality)

                if result['person_detected'] and result['faces_recognized']:
                    face_data = result['faces_recognized'][0]  # Take best face

                    # Quality validation
                    if face_data['det_score'] > 0.7:  # Good quality threshold
                        face_embeddings.append(face_data['embedding'])

                        # Collect metadata
                        metadata = {
                            'detection_confidence': face_data['det_score'],
                            'landmarks': face_data.get('landmarks'),
                            'stream_type': face_data.get('stream_type', 'gpu'),
                            'processing_time_ms': face_data.get('processing_time_ms', 0.0),
                            'image_metadata': {
                                'original_size': (width, height),
                                'processed_size': high_quality.shape[:2],
                                'file_path': img_path
                            }
                        }
                        registration_metadata.append(metadata)
                        processed_count += 1

                        print(f"    ✅ Confidence: {face_data['det_score']:.3f}, Stream: {face_data.get('stream_type', 'gpu')}")
                    else:
                        print(f"    ⚠️ Low quality: {face_data['det_score']:.3f}")
                elif result['person_detected']:
                    print(f"    ❌ Person detected but no face recognized")
                else:
                    print(f"    ❌ No person detected")

            except Exception as e:
                print(f"    ❌ Processing error: {e}")

        # Register employee if sufficient faces
        if len(face_embeddings) >= 1:
            # Create employee data
            employee_code = employee_name.upper().replace(' ', '_')
            employee_data = {
                'employee_code': employee_code,
                'name': employee_name,
                'email': f"{employee_code.lower()}@company.com",
                'department': 'Demo Department',
                'position': 'Employee'
            }

            # Register in database
            employee_id = self.db.register_employee(
                employee_data,
                face_embeddings,
                registration_metadata
            )

            if employee_id:
                avg_quality = np.mean([m.get('detection_confidence', 0.0) for m in registration_metadata])
                self.registered_employees.append({
                    'id': employee_id,
                    'name': employee_name,
                    'face_count': len(face_embeddings),
                    'avg_quality': avg_quality,
                    'processing_method': 'dual_stream'
                })

                print(f"  🎉 REGISTERED SUCCESSFULLY!")
                print(f"    ├─ Employee ID: {employee_id}")
                print(f"    ├─ Face embeddings: {len(face_embeddings)}")
                print(f"    ├─ Average quality: {avg_quality:.3f}")
                print(f"    └─ Processing: Dual-stream")
            else:
                print(f"  ❌ Database registration failed")
        else:
            print(f"  ❌ Insufficient quality faces (need at least 1)")
            print(f"    └─ Try uploading clearer, well-lit photos")

    def show_registered_employees(self):
        """Display registered employees with comprehensive information"""
        print("\n📋 REGISTERED EMPLOYEES")
        print("=" * 30)

        # Get all employees from database
        cursor = self.db.conn.cursor()
        cursor.execute("""
        SELECT id, employee_code, name, email, department, position,
               embedding_count, registration_quality, created_at
        FROM employees WHERE is_active = 1
        ORDER BY created_at DESC
        """)

        employees = [dict(row) for row in cursor.fetchall()]

        if not employees:
            print("📝 No employees registered yet")
            print("💡 Use employee_manager.upload_employee_photos() to register employees")
            return

        # Display in table format
        print(f"Total Employees: {len(employees)}")
        print()

        # Create DataFrame for better display
        df_data = []
        for emp in employees:
            df_data.append({
                'ID': emp['id'],
                'Name': emp['name'],
                'Code': emp['employee_code'],
                'Department': emp['department'],
                'Faces': emp['embedding_count'],
                'Quality': f"{emp['registration_quality']:.3f}",
                'Registered': emp['created_at'][:19]  # Remove microseconds
            })

        df = pd.DataFrame(df_data)

        if COLAB_ENV:
            display(HTML(df.to_html(index=False)))
        else:
            print(df.to_string(index=False))

        # Show summary statistics
        total_faces = sum(emp['embedding_count'] for emp in employees)
        avg_quality = np.mean([emp['registration_quality'] for emp in employees if emp['registration_quality'] > 0])

        print(f"\n📊 SUMMARY:")
        print(f"├─ Total Employees: {len(employees)}")
        print(f"├─ Total Face Images: {total_faces}")
        print(f"├─ Average Quality: {avg_quality:.3f}")
        print(f"└─ Registration Method: Dual-stream processing")

    def get_employee_statistics(self) -> Dict:
        """Get detailed employee statistics"""
        return self.db.get_statistics()

    def create_sample_employee_data(self):
        """Create sample employee data for testing"""
        print("\n🧪 CREATING SAMPLE EMPLOYEE DATA")
        print("=" * 35)

        sample_employees = [
            {
                'employee_code': 'EMP001',
                'name': 'John Smith',
                'email': 'john.smith@company.com',
                'department': 'Engineering',
                'position': 'Senior Developer'
            },
            {
                'employee_code': 'EMP002',
                'name': 'Sarah Johnson',
                'email': 'sarah.johnson@company.com',
                'department': 'Marketing',
                'position': 'Marketing Manager'
            },
            {
                'employee_code': 'EMP003',
                'name': 'Mike Wilson',
                'email': 'mike.wilson@company.com',
                'department': 'Sales',
                'position': 'Sales Representative'
            }
        ]

        import random

        for emp_data in sample_employees:
            # Generate random face embeddings (simulating real photos)
            embeddings = []
            metadata = []

            for i in range(random.randint(3, 5)):  # 3-5 face images per employee
                # Generate normalized random embedding
                embedding = np.random.normal(0, 1, 512)
                embedding = embedding / np.linalg.norm(embedding)
                embeddings.append(embedding)

                # Create realistic metadata
                meta = {
                    'detection_confidence': random.uniform(0.75, 0.95),
                    'landmarks': np.random.rand(5, 2).tolist(),
                    'stream_type': 'gpu',
                    'processing_time_ms': random.uniform(50, 150),
                    'image_metadata': {
                        'original_size': (1024, 768),
                        'processed_size': (640, 480),
                        'file_path': f"sample_{emp_data['employee_code']}_{i}.jpg"
                    }
                }
                metadata.append(meta)

            # Register employee
            employee_id = self.db.register_employee(emp_data, embeddings, metadata)
            if employee_id:
                print(f"✅ {emp_data['name']}: ID {employee_id}, {len(embeddings)} embeddings")

        print(f"\n🎉 Sample employee data created!")

# Initialize when imported
if __name__ == "__main__":
    print("✅ Cell 4 completed. Employee Management System ready.")
    print("💡 Use with ai_system, database, and config from previous cells.")

✅ Cell 4 completed. Employee Management System ready.
💡 Use with ai_system, database, and config from previous cells.


In [5]:
# -*- coding: utf-8 -*-
"""
CELL 5: VIDEO PROCESSING SYSTEM
- Upload and process video files with dual-stream processing
- Frame-by-frame attendance tracking with business logic
- Real-time analytics and progress tracking
- Interactive video upload with progress visualization
"""

import cv2
import numpy as np
import time
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple, Iterator
from pathlib import Path
import json
from collections import defaultdict, deque
import pandas as pd
from tqdm import tqdm

# Check if running in Colab
try:
    from google.colab import files
    from IPython.display import display, HTML, clear_output
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class VideoProcessor:
    """
    Video Processing System for Pipeline V1
    Handles video upload, dual-stream processing, and attendance tracking
    """

    def __init__(self, ai_system, database, config: Dict):
        self.ai_system = ai_system
        self.db = database
        self.config = config

        # Business logic configuration
        self.business_config = {
            'cooldown_minutes': 30,
            'work_hours_start': '07:00',
            'work_hours_end': '19:00',
            'recognition_threshold': 0.65,
            'processing_fps': 5,  # Process 5 frames per second
            'min_confidence': 0.7
        }

        # Processing statistics
        self.processing_stats = {
            'total_frames': 0,
            'frames_processed': 0,
            'persons_detected': 0,
            'faces_recognized': 0,
            'attendance_recorded': 0,
            'processing_times': deque(maxlen=100),
            'recognition_history': [],
            'session_start': time.time()
        }

        # Attendance events for this session
        self.attendance_events = []

        # Employee cooldown tracking (employee_id -> last_seen_time)
        self.employee_cooldowns = {}

        print("\n🎥 CELL 5: VIDEO PROCESSING SYSTEM")
        print("=" * 40)
        print("├─ Dual-Stream Processing: CPU detection + GPU recognition")
        print("├─ Business Logic: 30min cooldown, work hours validation")
        print("├─ Real-time Analytics: Frame-by-frame tracking")
        print("└─ Interactive Upload: Progress visualization")

    def upload_and_process_video(self):
        """Upload and process video file with Colab integration"""
        if not COLAB_ENV:
            print("❌ This function requires Google Colab environment")
            return

        print("📤 VIDEO UPLOAD & PROCESSING")
        print("=" * 35)
        print("📋 Supported formats: MP4, AVI, MOV, MKV")
        print("💡 Recommended: 30fps, 1080p or lower for optimal processing")

        try:
            uploaded = files.upload()

            for filename, content in uploaded.items():
                print(f"\n🎬 Processing video: {filename}")

                # Save uploaded file
                video_path = self.config['videos_dir'] / filename
                with open(video_path, 'wb') as f:
                    f.write(content)

                # Process video
                self.process_video_file(video_path, filename)

                # Cleanup
                if video_path.exists():
                    video_path.unlink()

        except Exception as e:
            logger.error(f"Video upload error: {e}")

    def process_video_file(self, video_path: Path, video_name: str):
        """Process video file with dual-stream Pipeline V1"""
        print(f"\n📊 VIDEO ANALYSIS: {video_name}")
        print("=" * 50)

        # Reset processing stats for this video
        self._reset_processing_stats()

        cap = cv2.VideoCapture(str(video_path))

        if not cap.isOpened():
            print("❌ Cannot open video file")
            return

        # Video properties
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0

        print(f"📹 Video Properties:")
        print(f"├─ Total Frames: {total_frames:,}")
        print(f"├─ FPS: {fps:.1f}")
        print(f"├─ Duration: {duration:.1f} seconds")
        print(f"└─ Processing Rate: {self.business_config['processing_fps']} FPS")

        # Calculate processing interval
        frame_interval = max(1, int(fps // self.business_config['processing_fps']))
        frames_to_process = total_frames // frame_interval

        print(f"\n🔄 Starting dual-stream processing...")
        print(f"├─ Frame interval: Every {frame_interval} frames")
        print(f"└─ Estimated frames to process: {frames_to_process:,}")

        # Process frames with progress tracking
        frame_count = 0
        processed_count = 0

        with tqdm(total=frames_to_process, desc="Processing video") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1
                self.processing_stats['total_frames'] = frame_count

                # Process only at specified intervals
                if frame_count % frame_interval == 0:
                    processed_count += 1

                    # Calculate timestamp in video
                    video_timestamp = frame_count / fps

                    # Process frame with Pipeline V1
                    results = self._process_frame_pipeline_v1(
                        frame, frame_count, video_name, video_timestamp
                    )

                    # Update progress
                    pbar.update(1)
                    pbar.set_postfix({
                        'Detected': self.processing_stats['persons_detected'],
                        'Recognized': self.processing_stats['faces_recognized'],
                        'Attendance': self.processing_stats['attendance_recorded']
                    })

        cap.release()

        # Generate comprehensive analytics
        self._generate_video_analytics(video_name, duration, fps)

    def _process_frame_pipeline_v1(self, frame: np.ndarray, frame_number: int,
                                  video_name: str, video_timestamp: float) -> Dict:
        """Process single frame using Pipeline V1 dual-stream architecture"""
        start_time = time.time()

        result = {
            'frame_number': frame_number,
            'video_timestamp': video_timestamp,
            'person_detected': False,
            'faces_recognized': [],
            'attendance_events': [],
            'processing_time_ms': 0.0
        }

        try:
            # Step 1: Create dual-stream frames
            height, width = frame.shape[:2]

            # Low quality for CPU person detection (Pipeline V1)
            low_quality_frame = cv2.resize(frame, (320, 240))

            # High quality for GPU face recognition
            if width > 640 or height > 480:
                scale = min(640/width, 480/height)
                new_width = int(width * scale)
                new_height = int(height * scale)
                high_quality_frame = cv2.resize(frame, (new_width, new_height))
            else:
                high_quality_frame = frame.copy()

            # Step 2: Dual-stream processing
            dual_stream_result = self.ai_system.process_dual_stream(
                low_quality_frame, high_quality_frame
            )

            result['person_detected'] = dual_stream_result['person_detected']

            if dual_stream_result['person_detected']:
                self.processing_stats['persons_detected'] += 1

                # Step 3: Process recognized faces
                for face_data in dual_stream_result['faces_recognized']:
                    # Try to recognize employee
                    employee = self.db.find_employee_by_embedding(
                        face_data['embedding'],
                        self.business_config['recognition_threshold']
                    )

                    if employee and face_data['det_score'] >= self.business_config['min_confidence']:
                        self.processing_stats['faces_recognized'] += 1

                        # Step 4: Apply business logic
                        attendance_decision = self._apply_business_logic(
                            employee['id'], frame_number, video_name, video_timestamp
                        )

                        face_result = {
                            'employee_id': employee['id'],
                            'employee_name': employee['name'],
                            'employee_code': employee['employee_code'],
                            'similarity': employee['similarity'],
                            'detection_confidence': face_data['det_score'],
                            'business_decision': attendance_decision
                        }
                        result['faces_recognized'].append(face_result)

                        # Step 5: Record attendance if approved
                        if attendance_decision['should_record']:
                            attendance_event = self._record_video_attendance(
                                employee, face_data, frame_number, video_name,
                                video_timestamp, attendance_decision
                            )
                            if attendance_event:
                                result['attendance_events'].append(attendance_event)
                                self.attendance_events.append(attendance_event)
                                self.processing_stats['attendance_recorded'] += 1

        except Exception as e:
            logger.error(f"Frame processing error: {e}")

        # Update processing statistics
        processing_time = (time.time() - start_time) * 1000
        result['processing_time_ms'] = processing_time
        self.processing_stats['processing_times'].append(processing_time)
        self.processing_stats['frames_processed'] += 1

        return result

    def _apply_business_logic(self, employee_id: int, frame_number: int,
                            video_name: str, video_timestamp: float) -> Dict:
        """Apply Pipeline V1 business logic for attendance decisions"""

        decision = {
            'should_record': False,
            'reason': '',
            'event_type': None,
            'checks': {
                'work_hours': False,
                'cooldown': False,
                'existing_records': False
            }
        }

        try:
            # Get current time (simulated from video timestamp)
            # In real implementation, this would be actual current time
            current_time = datetime.now()

            # 1. Work hours check
            work_start = datetime.strptime(self.business_config['work_hours_start'], '%H:%M').time()
            work_end = datetime.strptime(self.business_config['work_hours_end'], '%H:%M').time()
            current_time_only = current_time.time()

            in_work_hours = work_start <= current_time_only <= work_end
            decision['checks']['work_hours'] = in_work_hours

            if not in_work_hours:
                decision['reason'] = f'Outside work hours ({self.business_config["work_hours_start"]}-{self.business_config["work_hours_end"]})'
                return decision

            # 2. Cooldown check (using session-based tracking for video processing)
            current_timestamp = time.time()
            last_seen = self.employee_cooldowns.get(employee_id, 0)
            time_since_last = (current_timestamp - last_seen) / 60  # minutes

            cooldown_ok = time_since_last >= self.business_config['cooldown_minutes']
            decision['checks']['cooldown'] = cooldown_ok

            if not cooldown_ok:
                remaining_minutes = self.business_config['cooldown_minutes'] - time_since_last
                decision['reason'] = f'Cooldown period: {remaining_minutes:.1f} minutes remaining'
                return decision

            # 3. Check existing records for event type determination
            today_records = self.db.get_today_records(employee_id)
            decision['checks']['existing_records'] = True

            # Determine event type
            if not today_records:
                event_type = 'check_in'
            else:
                last_event = today_records[-1]['event_type']
                event_type = 'check_out' if last_event == 'check_in' else 'check_in'

            # All checks passed
            decision['should_record'] = True
            decision['event_type'] = event_type
            decision['reason'] = f'Approved for {event_type}'

            # Update cooldown tracking
            self.employee_cooldowns[employee_id] = current_timestamp

        except Exception as e:
            decision['reason'] = f'Business logic error: {str(e)}'
            logger.error(f"Business logic error: {e}")

        return decision

    def _record_video_attendance(self, employee: Dict, face_data: Dict,
                               frame_number: int, video_name: str,
                               video_timestamp: float, business_decision: Dict) -> Optional[Dict]:
        """Record attendance event with comprehensive metadata"""

        try:
            # Prepare dual-stream metadata
            dual_stream_metadata = {
                'detection_stream': 'cpu',
                'recognition_stream': face_data.get('stream_type', 'gpu'),
                'cpu_processing_time_ms': 0.0,  # From dual-stream result
                'gpu_processing_time_ms': face_data.get('processing_time_ms', 0.0)
            }

            # Record attendance in database
            attendance_id = self.db.record_attendance(
                employee_id=employee['id'],
                event_type=business_decision['event_type'],
                confidence=employee['similarity'],
                dual_stream_metadata=dual_stream_metadata,
                video_source=video_name,
                frame_number=frame_number,
                face_bbox=face_data.get('bbox', []),
                face_quality=face_data.get('det_score', 0.0),
                face_landmarks=face_data.get('landmarks', []),
                zone_detected='video_processing',
                cooldown_applied=False,
                work_hours_valid=business_decision['checks']['work_hours']
            )

            if attendance_id:
                attendance_event = {
                    'attendance_id': attendance_id,
                    'employee_id': employee['id'],
                    'employee_name': employee['name'],
                    'employee_code': employee['employee_code'],
                    'event_type': business_decision['event_type'],
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'confidence': employee['similarity'],
                    'frame_number': frame_number,
                    'video_timestamp': video_timestamp,
                    'video_source': video_name
                }

                print(f"✅ Frame {frame_number}: {employee['name']} - {business_decision['event_type']} ({employee['similarity']:.3f})")

                return attendance_event

        except Exception as e:
            logger.error(f"Failed to record attendance: {e}")

        return None

    def _generate_video_analytics(self, video_name: str, duration: float, fps: float):
        """Generate comprehensive video processing analytics"""
        print(f"\n📊 VIDEO PROCESSING ANALYTICS")
        print("=" * 40)

        # Processing statistics
        avg_processing_time = np.mean(self.processing_stats['processing_times']) if self.processing_stats['processing_times'] else 0
        total_processing_time = sum(self.processing_stats['processing_times']) / 1000  # seconds

        print(f"📈 Processing Statistics:")
        print(f"├─ Total Frames: {self.processing_stats['total_frames']:,}")
        print(f"├─ Frames Processed: {self.processing_stats['frames_processed']:,}")
        print(f"├─ Persons Detected: {self.processing_stats['persons_detected']:,}")
        print(f"├─ Faces Recognized: {self.processing_stats['faces_recognized']:,}")
        print(f"├─ Attendance Recorded: {self.processing_stats['attendance_recorded']:,}")
        print(f"├─ Avg Processing Time: {avg_processing_time:.1f}ms/frame")
        print(f"├─ Total Processing Time: {total_processing_time:.1f}s")
        print(f"└─ Processing Efficiency: {self.processing_stats['frames_processed']/self.processing_stats['total_frames']*100:.1f}%")

        # Attendance summary
        if self.attendance_events:
            self._show_attendance_summary()
            if COLAB_ENV:
                self._plot_video_analytics(video_name, duration)
        else:
            print("\n⚠️ No attendance events recorded")
            print("💡 Possible reasons:")
            print("   ├─ No employees registered")
            print("   ├─ Low video quality")
            print("   ├─ Recognition threshold too high")
            print("   └─ Business logic restrictions")

    def _show_attendance_summary(self):
        """Show attendance summary table"""
        print(f"\n👥 ATTENDANCE SUMMARY")
        print("-" * 30)

        # Group by employee
        employee_summary = defaultdict(list)
        for event in self.attendance_events:
            employee_summary[event['employee_name']].append(event)

        summary_data = []
        for employee_name, events in employee_summary.items():
            first_event = min(events, key=lambda x: x['frame_number'])
            last_event = max(events, key=lambda x: x['frame_number'])
            avg_confidence = np.mean([e['confidence'] for e in events])

            summary_data.append({
                'Employee': employee_name,
                'First Detection': f"Frame {first_event['frame_number']} ({first_event['event_type']})",
                'Last Detection': f"Frame {last_event['frame_number']} ({last_event['event_type']})",
                'Total Events': len(events),
                'Avg Confidence': f"{avg_confidence:.3f}"
            })

        # Display as DataFrame
        df = pd.DataFrame(summary_data)
        if COLAB_ENV:
            display(HTML(df.to_html(index=False)))
        else:
            print(df.to_string(index=False))

    def _plot_video_analytics(self, video_name: str, duration: float):
        """Plot interactive video processing analytics"""
        if not COLAB_ENV:
            return

        print(f"\n📈 INTERACTIVE ANALYTICS")

        # Create timeline plot
        fig = make_subplots(
            rows=3, cols=1,
            subplot_titles=(
                'Attendance Timeline',
                'Processing Performance',
                'Recognition Confidence Distribution'
            ),
            vertical_spacing=0.1
        )

        # 1. Attendance Timeline
        if self.attendance_events:
            for employee_name in set(e['employee_name'] for e in self.attendance_events):
                employee_events = [e for e in self.attendance_events if e['employee_name'] == employee_name]
                frame_numbers = [e['frame_number'] for e in employee_events]
                video_timestamps = [e['video_timestamp'] for e in employee_events]
                confidences = [e['confidence'] for e in employee_events]

                fig.add_trace(
                    go.Scatter(
                        x=video_timestamps,
                        y=confidences,
                        mode='markers+lines',
                        name=employee_name,
                        text=[f"Frame {f}" for f in frame_numbers],
                        hovertemplate='<b>%{fullData.name}</b><br>Time: %{x:.1f}s<br>Confidence: %{y:.3f}<br>%{text}<extra></extra>'
                    ),
                    row=1, col=1
                )

        # 2. Processing Performance
        if self.processing_stats['processing_times']:
            frame_indices = list(range(len(self.processing_stats['processing_times'])))
            processing_times = list(self.processing_stats['processing_times'])

            fig.add_trace(
                go.Scatter(
                    x=frame_indices,
                    y=processing_times,
                    mode='lines',
                    name='Processing Time',
                    line=dict(color='orange'),
                    showlegend=False
                ),
                row=2, col=1
            )

        # 3. Confidence Distribution
        if self.attendance_events:
            confidences = [e['confidence'] for e in self.attendance_events]
            fig.add_trace(
                go.Histogram(
                    x=confidences,
                    nbinsx=20,
                    name='Confidence Distribution',
                    marker_color='lightblue',
                    showlegend=False
                ),
                row=3, col=1
            )

        # Update layout
        fig.update_layout(
            height=800,
            title_text=f"Video Processing Analytics: {video_name}",
            showlegend=True
        )

        fig.update_xaxes(title_text="Video Time (seconds)", row=1, col=1)
        fig.update_yaxes(title_text="Recognition Confidence", row=1, col=1)
        fig.update_xaxes(title_text="Processed Frame Index", row=2, col=1)
        fig.update_yaxes(title_text="Processing Time (ms)", row=2, col=1)
        fig.update_xaxes(title_text="Confidence Score", row=3, col=1)
        fig.update_yaxes(title_text="Frequency", row=3, col=1)

        fig.show()

    def _reset_processing_stats(self):
        """Reset processing statistics for new video"""
        self.processing_stats = {
            'total_frames': 0,
            'frames_processed': 0,
            'persons_detected': 0,
            'faces_recognized': 0,
            'attendance_recorded': 0,
            'processing_times': deque(maxlen=100),
            'recognition_history': [],
            'session_start': time.time()
        }
        self.attendance_events = []
        self.employee_cooldowns = {}

    def get_processing_summary(self) -> Dict:
        """Get comprehensive processing summary"""
        return {
            'processing_stats': dict(self.processing_stats),
            'attendance_events': self.attendance_events,
            'business_config': self.business_config,
            'employee_cooldowns': self.employee_cooldowns
        }

# Initialize when imported
if __name__ == "__main__":
    print("✅ Cell 5 completed. Video Processing System ready.")
    print("💡 Use with ai_system, database, and config from previous cells.")

✅ Cell 5 completed. Video Processing System ready.
💡 Use with ai_system, database, and config from previous cells.


In [6]:
# -*- coding: utf-8 -*-
"""
CELL 6: ANALYTICS DASHBOARD & REPORTING
- Real-time system performance monitoring
- Comprehensive attendance analytics
- Interactive dashboards with Plotly
- Multi-format export functionality (CSV, Excel, JSON)
- System health monitoring and optimization
"""

import numpy as np
import pandas as pd
import time
import json
from datetime import datetime, timedelta
from typing import Dict, List, Optional
from pathlib import Path
import logging

# Check if running in Colab
try:
    from google.colab import files
    from IPython.display import display, HTML, clear_output
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    import ipywidgets as widgets
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class AnalyticsDashboard:
    """
    Analytics Dashboard & Reporting System for Pipeline V1
    Comprehensive monitoring and reporting capabilities
    """

    def __init__(self, ai_system, database, video_processor, config: Dict):
        self.ai_system = ai_system
        self.db = database
        self.video_processor = video_processor
        self.config = config

        # Dashboard state
        self.dashboard_data = {
            'last_update': datetime.now(),
            'system_metrics': [],
            'performance_history': []
        }

        print("\n📊 CELL 6: ANALYTICS DASHBOARD & REPORTING")
        print("=" * 45)
        print("├─ Real-time Performance Monitoring")
        print("├─ Attendance Analytics & Insights")
        print("├─ Interactive Plotly Dashboards")
        print("├─ Multi-format Export (CSV, Excel, JSON)")
        print("└─ System Health & Optimization")

    def show_system_dashboard(self):
        """Display comprehensive real-time system dashboard"""
        print("🚀 SYSTEM PERFORMANCE DASHBOARD")
        print("=" * 40)

        # Collect current metrics
        system_metrics = self._collect_system_metrics()

        # Show system status
        self._show_system_status(system_metrics)

        # Show AI performance
        self._show_ai_performance()

        # Show database analytics
        self._show_database_analytics()

        # Interactive performance visualization
        if COLAB_ENV:
            self._create_performance_dashboard()

    def _collect_system_metrics(self) -> Dict:
        """Collect comprehensive system metrics"""
        metrics = {
            'timestamp': datetime.now(),
            'system_info': {},
            'ai_performance': {},
            'database_stats': {},
            'processing_stats': {}
        }

        try:
            # System information
            import psutil
            import torch

            metrics['system_info'] = {
                'cpu_usage': psutil.cpu_percent(),
                'memory_usage': psutil.virtual_memory().percent,
                'memory_available_gb': psutil.virtual_memory().available / (1024**3),
                'gpu_available': torch.cuda.is_available()
            }

            if torch.cuda.is_available():
                metrics['system_info'].update({
                    'gpu_memory_used_mb': torch.cuda.memory_allocated() / (1024**2),
                    'gpu_memory_total_mb': torch.cuda.get_device_properties(0).total_memory / (1024**2),
                    'gpu_name': torch.cuda.get_device_name(0)
                })

            # AI Performance
            metrics['ai_performance'] = self.ai_system.get_performance_stats()

            # Database statistics
            metrics['database_stats'] = self.db.get_statistics()

            # Video processing statistics
            if hasattr(self.video_processor, 'get_processing_summary'):
                metrics['processing_stats'] = self.video_processor.get_processing_summary()

        except Exception as e:
            logger.error(f"Error collecting system metrics: {e}")

        # Store metrics history
        self.dashboard_data['system_metrics'].append(metrics)
        if len(self.dashboard_data['system_metrics']) > 100:  # Keep last 100 metrics
            self.dashboard_data['system_metrics'] = self.dashboard_data['system_metrics'][-100:]

        return metrics

    def _show_system_status(self, metrics: Dict):
        """Display current system status"""
        print(f"\n🖥️ SYSTEM STATUS")
        print("-" * 25)

        sys_info = metrics['system_info']
        print(f"├─ CPU Usage: {sys_info.get('cpu_usage', 0):.1f}%")
        print(f"├─ Memory Usage: {sys_info.get('memory_usage', 0):.1f}%")
        print(f"├─ Memory Available: {sys_info.get('memory_available_gb', 0):.2f} GB")
        print(f"├─ GPU Available: {sys_info.get('gpu_available', False)}")

        if sys_info.get('gpu_available'):
            print(f"├─ GPU: {sys_info.get('gpu_name', 'Unknown')}")
            print(f"├─ GPU Memory Used: {sys_info.get('gpu_memory_used_mb', 0):.1f} MB")
            print(f"└─ GPU Memory Total: {sys_info.get('gpu_memory_total_mb', 0):.1f} MB")
        else:
            print(f"└─ GPU: Not Available")

    def _show_ai_performance(self):
        """Display AI model performance metrics"""
        ai_stats = self.ai_system.get_performance_stats()

        print(f"\n🤖 AI MODEL PERFORMANCE")
        print("-" * 30)
        print(f"├─ Total Inferences: {ai_stats.get('total_inferences', 0):,}")
        print(f"├─ CPU Detections: {ai_stats.get('cpu_detections', 0):,}")
        print(f"├─ GPU Recognitions: {ai_stats.get('gpu_recognitions', 0):,}")
        print(f"├─ Avg CPU Latency: {ai_stats.get('avg_cpu_latency_ms', 0):.1f}ms")
        print(f"├─ Avg GPU Latency: {ai_stats.get('avg_gpu_latency_ms', 0):.1f}ms")
        print(f"├─ Dual Stream: {ai_stats.get('dual_stream_enabled', False)}")
        print(f"└─ Model Pack: {ai_stats.get('model_pack', 'Unknown')}")

        # Performance rating
        total_latency = ai_stats.get('avg_cpu_latency_ms', 0) + ai_stats.get('avg_gpu_latency_ms', 0)
        if total_latency < 100:
            rating = "🌟 EXCELLENT"
        elif total_latency < 200:
            rating = "👍 GOOD"
        elif total_latency < 400:
            rating = "⚠️ MODERATE"
        else:
            rating = "🐌 NEEDS OPTIMIZATION"

        print(f"   └─ Performance Rating: {rating}")

    def _show_database_analytics(self):
        """Display database analytics and statistics"""
        db_stats = self.db.get_statistics()

        print(f"\n🗄️ DATABASE ANALYTICS")
        print("-" * 25)
        print(f"├─ Active Employees: {db_stats.get('total_active_employees', 0)}")
        print(f"├─ Employees with Faces: {db_stats.get('employees_with_faces', 0)}")
        print(f"├─ Total Attendance Logs: {db_stats.get('total_attendance_logs', 0)}")
        print(f"├─ Face Registrations: {db_stats.get('total_face_registrations', 0)}")
        print(f"├─ Today's Attendance: {db_stats.get('todays_attendance_count', 0)}")
        print(f"├─ Avg Registration Quality: {db_stats.get('avg_registration_quality', 0):.3f}")
        print(f"├─ CPU Detections: {db_stats.get('cpu_detections', 0)}")
        print(f"├─ GPU Recognitions: {db_stats.get('gpu_recognitions', 0)}")
        print(f"└─ Last 24h Activity: {db_stats.get('activity_last_24h', 0)}")

    def _create_performance_dashboard(self):
        """Create interactive performance dashboard with Plotly"""
        if not COLAB_ENV:
            return

        print(f"\n📈 INTERACTIVE PERFORMANCE DASHBOARD")

        # Create comprehensive dashboard
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                'System Resource Usage', 'AI Processing Performance',
                'Database Statistics', 'Processing Timeline',
                'Recognition Accuracy Distribution', 'System Health Score'
            ),
            specs=[[{"secondary_y": True}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "indicator"}]],
            vertical_spacing=0.08
        )

        # 1. System Resource Usage
        if len(self.dashboard_data['system_metrics']) > 1:
            timestamps = [m['timestamp'] for m in self.dashboard_data['system_metrics'][-20:]]
            cpu_usage = [m['system_info'].get('cpu_usage', 0) for m in self.dashboard_data['system_metrics'][-20:]]
            memory_usage = [m['system_info'].get('memory_usage', 0) for m in self.dashboard_data['system_metrics'][-20:]]

            fig.add_trace(
                go.Scatter(x=timestamps, y=cpu_usage, name='CPU %', line=dict(color='blue')),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatter(x=timestamps, y=memory_usage, name='Memory %', line=dict(color='red')),
                row=1, col=1, secondary_y=True
            )

        # 2. AI Processing Performance
        ai_stats = self.ai_system.get_performance_stats()
        ai_metrics = ['CPU Detections', 'GPU Recognitions', 'Total Inferences']
        ai_values = [
            ai_stats.get('cpu_detections', 0),
            ai_stats.get('gpu_recognitions', 0),
            ai_stats.get('total_inferences', 0)
        ]

        fig.add_trace(
            go.Bar(x=ai_metrics, y=ai_values, name='AI Performance', marker_color='green'),
            row=1, col=2
        )

        # 3. Database Statistics
        db_stats = self.db.get_statistics()
        db_labels = ['Employees', 'Attendance Logs', 'Face Registrations', 'Today\'s Logs']
        db_values = [
            db_stats.get('total_active_employees', 0),
            db_stats.get('total_attendance_logs', 0),
            db_stats.get('total_face_registrations', 0),
            db_stats.get('todays_attendance_count', 0)
        ]

        fig.add_trace(
            go.Bar(x=db_labels, y=db_values, name='Database Stats', marker_color='purple'),
            row=2, col=1
        )

        # 4. Processing Timeline (if video processing occurred)
        if hasattr(self.video_processor, 'processing_stats') and self.video_processor.processing_stats.get('processing_times'):
            processing_times = list(self.video_processor.processing_stats['processing_times'])
            frame_numbers = list(range(len(processing_times)))

            fig.add_trace(
                go.Scatter(x=frame_numbers, y=processing_times, name='Frame Processing Time',
                          line=dict(color='orange')),
                row=2, col=2
            )

        # 5. Recognition Accuracy Distribution
        attendance_logs = self.db.get_attendance_logs(limit=100)
        if attendance_logs:
            confidences = [log['confidence'] for log in attendance_logs if log.get('confidence')]

            fig.add_trace(
                go.Histogram(x=confidences, nbinsx=15, name='Recognition Confidence',
                           marker_color='lightblue'),
                row=3, col=1
            )

        # 6. System Health Score (Indicator)
        health_score = self._calculate_system_health_score()

        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=health_score,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': "System Health"},
                delta={'reference': 80},
                gauge={
                    'axis': {'range': [None, 100]},
                    'bar': {'color': "darkgreen"},
                    'steps': [
                        {'range': [0, 50], 'color': "lightgray"},
                        {'range': [50, 80], 'color': "yellow"},
                        {'range': [80, 100], 'color': "green"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 90
                    }
                }
            ),
            row=3, col=2
        )

        # Update layout
        fig.update_layout(
            height=900,
            title_text="🚀 AI Attendance System - Real-time Dashboard",
            showlegend=True
        )

        # Update axes labels
        fig.update_xaxes(title_text="Time", row=1, col=1)
        fig.update_yaxes(title_text="CPU Usage %", row=1, col=1)
        fig.update_yaxes(title_text="Memory Usage %", row=1, col=1, secondary_y=True)

        fig.show()

    def show_attendance_report(self, days_back: int = 7):
        """Display comprehensive attendance analytics"""
        print("📋 ATTENDANCE ANALYTICS REPORT")
        print("=" * 35)

        # Get attendance data
        cursor = self.db.conn.cursor()

        # Date range for analysis
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)

        cursor.execute("""
        SELECT al.*, e.name, e.employee_code, e.department
        FROM attendance_logs al
        JOIN employees e ON al.employee_id = e.id
        WHERE al.timestamp >= ?
        ORDER BY al.timestamp DESC
        """, (start_date.strftime('%Y-%m-%d %H:%M:%S'),))

        attendance_records = [dict(row) for row in cursor.fetchall()]

        if not attendance_records:
            print("📝 No attendance records found for the specified period")
            return

        # Basic statistics
        print(f"📊 SUMMARY ({days_back} days):")
        print(f"├─ Total Records: {len(attendance_records)}")
        print(f"├─ Unique Employees: {len(set(r['employee_id'] for r in attendance_records))}")
        print(f"├─ Check-ins: {len([r for r in attendance_records if r['event_type'] == 'check_in'])}")
        print(f"├─ Check-outs: {len([r for r in attendance_records if r['event_type'] == 'check_out'])}")
        print(f"└─ Avg Confidence: {np.mean([r['confidence'] for r in attendance_records]):.3f}")

        # Department breakdown
        dept_breakdown = {}
        for record in attendance_records:
            dept = record.get('department', 'Unknown')
            dept_breakdown[dept] = dept_breakdown.get(dept, 0) + 1

        print(f"\n🏢 DEPARTMENT BREAKDOWN:")
        for dept, count in sorted(dept_breakdown.items()):
            print(f"├─ {dept}: {count} records")

        # Daily attendance trends
        daily_counts = {}
        for record in attendance_records:
            date = record['timestamp'][:10]  # YYYY-MM-DD
            daily_counts[date] = daily_counts.get(date, 0) + 1

        print(f"\n📅 DAILY ATTENDANCE TRENDS:")
        for date in sorted(daily_counts.keys()):
            print(f"├─ {date}: {daily_counts[date]} records")

        # Create detailed DataFrame
        df = pd.DataFrame(attendance_records)

        if COLAB_ENV:
            # Display interactive table
            display(HTML(f"<h3>📋 Detailed Attendance Records</h3>"))
            display_df = df[['name', 'employee_code', 'department', 'event_type',
                           'timestamp', 'confidence']].head(20)
            display(HTML(display_df.to_html(index=False)))

            # Create attendance analytics charts
            self._create_attendance_charts(df, days_back)

    def _create_attendance_charts(self, df: pd.DataFrame, days_back: int):
        """Create interactive attendance analytics charts"""
        if not COLAB_ENV or df.empty:
            return

        print(f"\n📊 INTERACTIVE ATTENDANCE ANALYTICS")

        # Create comprehensive attendance dashboard
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Daily Attendance Trends',
                'Department Distribution',
                'Hourly Patterns',
                'Event Type Distribution'
            ),
            specs=[[{"secondary_y": False}, {"type": "pie"}],
                   [{"secondary_y": False}, {"type": "pie"}]]
        )

        # 1. Daily Attendance Trends
        df['date'] = pd.to_datetime(df['timestamp']).dt.date
        daily_counts = df.groupby('date').size().reset_index(name='count')

        fig.add_trace(
            go.Scatter(
                x=daily_counts['date'],
                y=daily_counts['count'],
                mode='lines+markers',
                name='Daily Attendance',
                line=dict(color='blue', width=3),
                marker=dict(size=8)
            ),
            row=1, col=1
        )

        # 2. Department Distribution
        dept_counts = df['department'].value_counts()

        fig.add_trace(
            go.Pie(
                labels=dept_counts.index,
                values=dept_counts.values,
                name="Department Distribution"
            ),
            row=1, col=2
        )

        # 3. Hourly Patterns
        df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
        hourly_counts = df.groupby('hour').size().reset_index(name='count')

        fig.add_trace(
            go.Bar(
                x=hourly_counts['hour'],
                y=hourly_counts['count'],
                name='Hourly Distribution',
                marker_color='green'
            ),
            row=2, col=1
        )

        # 4. Event Type Distribution
        event_counts = df['event_type'].value_counts()

        fig.add_trace(
            go.Pie(
                labels=event_counts.index,
                values=event_counts.values,
                name="Event Type Distribution"
            ),
            row=2, col=2
        )

        # Update layout
        fig.update_layout(
            height=800,
            title_text=f"📊 Attendance Analytics Dashboard ({days_back} days)",
            showlegend=True
        )

        # Update axes
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_yaxes(title_text="Number of Records", row=1, col=1)
        fig.update_xaxes(title_text="Hour of Day", row=2, col=1)
        fig.update_yaxes(title_text="Number of Records", row=2, col=1)

        fig.show()

    def export_comprehensive_reports(self):
        """Export comprehensive reports in multiple formats"""
        print("📤 COMPREHENSIVE REPORT EXPORT")
        print("=" * 35)

        try:
            # Collect all data for export
            export_data = self._prepare_export_data()

            # Export to different formats
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

            # 1. JSON Export (Complete system state)
            json_path = self.config['exports_dir'] / f'system_report_{timestamp}.json'
            with open(json_path, 'w') as f:
                json.dump(export_data, f, indent=2, default=str)
            print(f"✅ JSON Report: {json_path}")

            # 2. CSV Export (Attendance logs)
            if export_data['attendance_logs']:
                csv_path = self.config['exports_dir'] / f'attendance_logs_{timestamp}.csv'
                df_attendance = pd.DataFrame(export_data['attendance_logs'])
                df_attendance.to_csv(csv_path, index=False)
                print(f"✅ CSV Attendance: {csv_path}")

            # 3. Excel Export (Multiple sheets)
            excel_path = self.config['exports_dir'] / f'attendance_report_{timestamp}.xlsx'
            with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                # Attendance logs sheet
                if export_data['attendance_logs']:
                    df_attendance = pd.DataFrame(export_data['attendance_logs'])
                    df_attendance.to_excel(writer, sheet_name='Attendance_Logs', index=False)

                # Employee list sheet
                if export_data['employees']:
                    df_employees = pd.DataFrame(export_data['employees'])
                    df_employees.to_excel(writer, sheet_name='Employees', index=False)

                # System metrics sheet
                if export_data['system_metrics']:
                    df_metrics = pd.DataFrame([export_data['system_metrics']])
                    df_metrics.to_excel(writer, sheet_name='System_Metrics', index=False)

            print(f"✅ Excel Report: {excel_path}")

            # 4. Summary Report (TXT)
            summary_path = self.config['exports_dir'] / f'summary_report_{timestamp}.txt'
            self._create_summary_report(export_data, summary_path)
            print(f"✅ Summary Report: {summary_path}")

            # Download files in Colab
            if COLAB_ENV:
                try:
                    files.download(str(json_path))
                    if export_data['attendance_logs']:
                        files.download(str(csv_path))
                    files.download(str(excel_path))
                    files.download(str(summary_path))
                    print("📥 Files downloaded to your computer")
                except Exception as e:
                    print(f"💡 Files saved to exports directory: {e}")

            print(f"\n📊 Export Summary:")
            print(f"├─ Attendance Records: {len(export_data['attendance_logs'])}")
            print(f"├─ Employees: {len(export_data['employees'])}")
            print(f"├─ Export Formats: JSON, CSV, Excel, Summary")
            print(f"└─ Export Location: {self.config['exports_dir']}")

        except Exception as e:
            logger.error(f"Export error: {e}")

    def _prepare_export_data(self) -> Dict:
        """Prepare comprehensive data for export"""
        export_data = {
            'export_metadata': {
                'timestamp': datetime.now().isoformat(),
                'system_version': 'Pipeline V1',
                'export_type': 'comprehensive_report'
            },
            'system_metrics': {},
            'employees': [],
            'attendance_logs': [],
            'performance_data': {}
        }

        try:
            # System metrics
            current_metrics = self._collect_system_metrics()
            export_data['system_metrics'] = current_metrics

            # Employee data
            cursor = self.db.conn.cursor()
            cursor.execute("""
            SELECT id, employee_code, name, email, department, position,
                   embedding_count, registration_quality, created_at
            FROM employees WHERE is_active = 1
            """)
            export_data['employees'] = [dict(row) for row in cursor.fetchall()]

            # Attendance logs (last 30 days)
            thirty_days_ago = datetime.now() - timedelta(days=30)
            cursor.execute("""
            SELECT al.*, e.name, e.employee_code, e.department
            FROM attendance_logs al
            JOIN employees e ON al.employee_id = e.id
            WHERE al.timestamp >= ?
            ORDER BY al.timestamp DESC
            """, (thirty_days_ago.strftime('%Y-%m-%d %H:%M:%S'),))
            export_data['attendance_logs'] = [dict(row) for row in cursor.fetchall()]

            # Performance data
            export_data['performance_data'] = {
                'ai_performance': self.ai_system.get_performance_stats(),
                'database_stats': self.db.get_statistics()
            }

            if hasattr(self.video_processor, 'get_processing_summary'):
                export_data['performance_data']['video_processing'] = self.video_processor.get_processing_summary()

        except Exception as e:
            logger.error(f"Error preparing export data: {e}")

        return export_data

    def _create_summary_report(self, export_data: Dict, file_path: Path):
        """Create human-readable summary report"""
        with open(file_path, 'w') as f:
            f.write("AI ATTENDANCE SYSTEM - COMPREHENSIVE REPORT\n")
            f.write("=" * 50 + "\n\n")

            # Report metadata
            f.write(f"Report Generated: {export_data['export_metadata']['timestamp']}\n")
            f.write(f"System Version: {export_data['export_metadata']['system_version']}\n\n")

            # System overview
            f.write("SYSTEM OVERVIEW\n")
            f.write("-" * 20 + "\n")

            metrics = export_data['system_metrics']
            if metrics:
                sys_info = metrics.get('system_info', {})
                f.write(f"CPU Usage: {sys_info.get('cpu_usage', 0):.1f}%\n")
                f.write(f"Memory Usage: {sys_info.get('memory_usage', 0):.1f}%\n")
                f.write(f"GPU Available: {sys_info.get('gpu_available', False)}\n")

                if sys_info.get('gpu_available'):
                    f.write(f"GPU: {sys_info.get('gpu_name', 'Unknown')}\n")

            f.write("\n")

            # Employee statistics
            f.write("EMPLOYEE STATISTICS\n")
            f.write("-" * 20 + "\n")
            f.write(f"Total Employees: {len(export_data['employees'])}\n")

            if export_data['employees']:
                dept_counts = {}
                for emp in export_data['employees']:
                    dept = emp.get('department', 'Unknown')
                    dept_counts[dept] = dept_counts.get(dept, 0) + 1

                f.write("Department Breakdown:\n")
                for dept, count in dept_counts.items():
                    f.write(f"  - {dept}: {count}\n")

            f.write("\n")

            # Attendance statistics
            f.write("ATTENDANCE STATISTICS\n")
            f.write("-" * 25 + "\n")
            f.write(f"Total Records: {len(export_data['attendance_logs'])}\n")

            if export_data['attendance_logs']:
                check_ins = len([r for r in export_data['attendance_logs'] if r['event_type'] == 'check_in'])
                check_outs = len([r for r in export_data['attendance_logs'] if r['event_type'] == 'check_out'])
                avg_confidence = np.mean([r['confidence'] for r in export_data['attendance_logs']])

                f.write(f"Check-ins: {check_ins}\n")
                f.write(f"Check-outs: {check_outs}\n")
                f.write(f"Average Confidence: {avg_confidence:.3f}\n")

            f.write("\n")

            # Performance summary
            f.write("PERFORMANCE SUMMARY\n")
            f.write("-" * 20 + "\n")

            perf_data = export_data['performance_data']
            if 'ai_performance' in perf_data:
                ai_perf = perf_data['ai_performance']
                f.write(f"Total AI Inferences: {ai_perf.get('total_inferences', 0)}\n")
                f.write(f"CPU Detections: {ai_perf.get('cpu_detections', 0)}\n")
                f.write(f"GPU Recognitions: {ai_perf.get('gpu_recognitions', 0)}\n")
                f.write(f"Average CPU Latency: {ai_perf.get('avg_cpu_latency_ms', 0):.1f}ms\n")
                f.write(f"Average GPU Latency: {ai_perf.get('avg_gpu_latency_ms', 0):.1f}ms\n")

    def _calculate_system_health_score(self) -> float:
        """Calculate overall system health score (0-100)"""
        try:
            score = 100.0

            # Get current metrics
            current_metrics = self._collect_system_metrics()
            sys_info = current_metrics['system_info']

            # CPU usage penalty
            cpu_usage = sys_info.get('cpu_usage', 0)
            if cpu_usage > 80:
                score -= (cpu_usage - 80) * 2

            # Memory usage penalty
            memory_usage = sys_info.get('memory_usage', 0)
            if memory_usage > 85:
                score -= (memory_usage - 85) * 3

            # AI performance bonus
            ai_stats = self.ai_system.get_performance_stats()
            total_latency = ai_stats.get('avg_cpu_latency_ms', 0) + ai_stats.get('avg_gpu_latency_ms', 0)
            if total_latency < 100:
                score += 10
            elif total_latency > 400:
                score -= 20

            # Database activity bonus
            db_stats = self.db.get_statistics()
            if db_stats.get('total_active_employees', 0) > 0:
                score += 5

            # GPU availability bonus
            if sys_info.get('gpu_available', False):
                score += 10

            return max(0.0, min(100.0, score))

        except Exception as e:
            logger.error(f"Error calculating health score: {e}")
            return 50.0  # Default moderate health score

    def create_interactive_widgets(self):
        """Create interactive widgets for dashboard control"""
        if not COLAB_ENV:
            print("❌ Interactive widgets require Google Colab environment")
            return

        print("🎛️ INTERACTIVE DASHBOARD CONTROLS")
        print("=" * 35)

        # Create control widgets
        refresh_button = widgets.Button(
            description='🔄 Refresh Dashboard',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px')
        )

        export_button = widgets.Button(
            description='📤 Export Reports',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )

        days_slider = widgets.IntSlider(
            value=7,
            min=1,
            max=30,
            step=1,
            description='Report Days:',
            style={'description_width': 'initial'}
        )

        output = widgets.Output()

        # Button event handlers
        def on_refresh_click(b):
            with output:
                clear_output()
                print("🔄 Refreshing dashboard...")
                self.show_system_dashboard()

        def on_export_click(b):
            with output:
                clear_output()
                print("📤 Exporting reports...")
                self.export_comprehensive_reports()

        def on_days_change(change):
            with output:
                clear_output()
                print(f"📊 Updating report for {change['new']} days...")
                self.show_attendance_report(change['new'])

        refresh_button.on_click(on_refresh_click)
        export_button.on_click(on_export_click)
        days_slider.observe(on_days_change, names='value')

        # Display widgets
        controls = widgets.HBox([refresh_button, export_button])
        display(widgets.VBox([controls, days_slider, output]))

# Initialize when imported
if __name__ == "__main__":
    print("✅ Cell 6 completed. Analytics Dashboard & Reporting ready.")
    print("💡 Use with ai_system, database, video_processor, and config from previous cells.")

✅ Cell 6 completed. Analytics Dashboard & Reporting ready.
💡 Use with ai_system, database, video_processor, and config from previous cells.


In [7]:
# -*- coding: utf-8 -*-
"""
CELL 7: MAIN EXECUTION & DEMO SYSTEM
- Complete system initialization and integration
- Sample data creation and demo scenarios
- Convenience wrapper functions for easy usage
- Interactive demo workflow
- System testing and validation
"""

import time
import logging
from datetime import datetime
from typing import Dict, Optional, Tuple
from pathlib import Path

# Check if running in Colab
try:
    from google.colab import files
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class PipelineV1System:
    """
    Complete AI Attendance System Pipeline V1
    Integrates all components with convenience functions
    """

    def __init__(self):
        # System components (will be initialized)
        self.config = None
        self.ai_system = None
        self.database = None
        self.employee_manager = None
        self.video_processor = None
        self.analytics_dashboard = None

        # System state
        self.is_initialized = False
        self.demo_mode = False

        print("\n🚀 CELL 7: AI ATTENDANCE SYSTEM PIPELINE V1")
        print("=" * 50)
        print("🎯 Complete system integration and demo capabilities")
        print("├─ Dual-Stream AI Processing")
        print("├─ SQLite Database with Vector Search")
        print("├─ Employee Management & Video Processing")
        print("├─ Analytics Dashboard & Reporting")
        print("└─ Interactive Demo & Testing")

    def initialize_system(self, model_pack='buffalo_l', demo_mode=True) -> bool:
        """Initialize complete Pipeline V1 system"""
        print(f"\n🔄 INITIALIZING PIPELINE V1 SYSTEM")
        print("=" * 40)
        print(f"├─ Model Pack: {model_pack}")
        print(f"├─ Demo Mode: {demo_mode}")
        print(f"└─ Environment: {'Google Colab' if COLAB_ENV else 'Local'}")

        try:
            # Import all required modules (assuming previous cells are loaded)
            global setup_system, DualStreamAISystem, AttendanceDatabaseSQLite
            global EmployeeManager, VideoProcessor, AnalyticsDashboard

            # Step 1: System setup
            print(f"\n📦 Step 1: System Setup...")
            self.config = setup_system()

            # Step 2: Initialize AI models
            print(f"\n🤖 Step 2: AI Models Initialization...")
            self.ai_system = DualStreamAISystem(model_pack=model_pack)

            # Step 3: Initialize database
            print(f"\n🗄️ Step 3: Database Initialization...")
            db_path = self.config['db_dir'] / 'attendance_pipeline_v1.db'
            self.database = AttendanceDatabaseSQLite(db_path=str(db_path))

            # Step 4: Initialize employee manager
            print(f"\n👥 Step 4: Employee Manager...")
            self.employee_manager = EmployeeManager(
                self.ai_system, self.database, self.config
            )

            # Step 5: Initialize video processor
            print(f"\n🎥 Step 5: Video Processor...")
            self.video_processor = VideoProcessor(
                self.ai_system, self.database, self.config
            )

            # Step 6: Initialize analytics dashboard
            print(f"\n📊 Step 6: Analytics Dashboard...")
            self.analytics_dashboard = AnalyticsDashboard(
                self.ai_system, self.database, self.video_processor, self.config
            )

            self.is_initialized = True
            self.demo_mode = demo_mode

            print(f"\n🎉 SYSTEM INITIALIZATION COMPLETED!")
            print(f"✅ All components ready for Pipeline V1 operations")

            # Create sample data in demo mode
            if demo_mode:
                print(f"\n🧪 Creating sample data for demo...")
                self.create_sample_data()

            return True

        except Exception as e:
            logger.error(f"❌ System initialization failed: {e}")
            print(f"💡 Make sure all previous cells (1-6) are executed")
            return False

    def create_sample_data(self):
        """Create comprehensive sample data for demo"""
        print(f"\n🧪 CREATING SAMPLE DATA FOR DEMO")
        print("-" * 35)

        try:
            # Create sample employees
            self.employee_manager.create_sample_employee_data()

            # Add some sample attendance records
            self._create_sample_attendance_records()

            print(f"✅ Sample data creation completed!")

        except Exception as e:
            logger.error(f"Sample data creation error: {e}")

    def _create_sample_attendance_records(self):
        """Create sample attendance records for demo"""
        import random
        from datetime import timedelta

        # Get all employees
        cursor = self.database.conn.cursor()
        cursor.execute("SELECT id, name FROM employees WHERE is_active = 1")
        employees = cursor.fetchall()

        if not employees:
            return

        # Create sample attendance for last few days
        base_time = datetime.now()

        for days_back in range(5):  # Last 5 days
            date = base_time - timedelta(days=days_back)

            for emp in employees:
                if random.random() > 0.3:  # 70% attendance rate
                    # Morning check-in
                    checkin_time = date.replace(
                        hour=random.randint(7, 9),
                        minute=random.randint(0, 59),
                        second=0, microsecond=0
                    )

                    # Create sample dual-stream metadata
                    dual_stream_metadata = {
                        'detection_stream': 'cpu',
                        'recognition_stream': 'gpu',
                        'cpu_processing_time_ms': random.uniform(30, 80),
                        'gpu_processing_time_ms': random.uniform(50, 150)
                    }

                    # Record check-in
                    self.database.record_attendance(
                        employee_id=emp['id'],
                        event_type='check_in',
                        confidence=random.uniform(0.75, 0.95),
                        timestamp=checkin_time.strftime('%Y-%m-%d %H:%M:%S'),
                        dual_stream_metadata=dual_stream_metadata,
                        video_source='demo_data',
                        face_quality=random.uniform(0.7, 0.9),
                        zone_detected='main_entrance'
                    )

                    # Evening check-out (80% chance)
                    if random.random() > 0.2:
                        checkout_time = checkin_time.replace(
                            hour=random.randint(17, 19),
                            minute=random.randint(0, 59)
                        )

                        self.database.record_attendance(
                            employee_id=emp['id'],
                            event_type='check_out',
                            confidence=random.uniform(0.75, 0.95),
                            timestamp=checkout_time.strftime('%Y-%m-%d %H:%M:%S'),
                            dual_stream_metadata=dual_stream_metadata,
                            video_source='demo_data',
                            face_quality=random.uniform(0.7, 0.9),
                            zone_detected='main_entrance'
                        )

        print(f"📋 Sample attendance records created for {len(employees)} employees")

    def run_interactive_demo(self):
        """Run interactive demo with step-by-step guidance"""
        if not self.is_initialized:
            print("❌ System not initialized. Run init() first.")
            return

        print(f"\n🎮 INTERACTIVE DEMO MODE")
        print("=" * 30)

        if COLAB_ENV:
            self._create_demo_interface()
        else:
            self._run_console_demo()

    def _create_demo_interface(self):
        """Create interactive demo interface for Colab"""
        print("🎛️ Interactive Demo Interface")

        # Demo action buttons
        demo_buttons = {
            'System Status': self.show_system_status,
            'Employee Management': self.demo_employee_management,
            'Video Processing': self.demo_video_processing,
            'Analytics Dashboard': self.demo_analytics,
            'Export Reports': self.demo_export
        }

        # Create buttons
        button_widgets = []
        for name, func in demo_buttons.items():
            button = widgets.Button(
                description=name,
                button_style='info',
                layout=widgets.Layout(width='200px', height='40px', margin='5px')
            )
            button.on_click(lambda b, f=func: self._execute_demo_action(f))
            button_widgets.append(button)

        # Output area
        output = widgets.Output()

        # Display interface
        button_box = widgets.VBox(button_widgets)
        demo_interface = widgets.HBox([button_box, output])

        display(widgets.VBox([
            widgets.HTML("<h3>🎮 AI Attendance System Demo Interface</h3>"),
            demo_interface
        ]))

        # Store output widget for demo actions
        self._demo_output = output

    def _execute_demo_action(self, action_func):
        """Execute demo action with output capture"""
        if hasattr(self, '_demo_output'):
            with self._demo_output:
                clear_output()
                action_func()

    def _run_console_demo(self):
        """Run console-based demo for non-Colab environments"""
        demo_options = {
            '1': ('System Status', self.show_system_status),
            '2': ('Employee Management', self.demo_employee_management),
            '3': ('Video Processing', self.demo_video_processing),
            '4': ('Analytics Dashboard', self.demo_analytics),
            '5': ('Export Reports', self.demo_export),
            '6': ('Exit Demo', None)
        }

        while True:
            print(f"\n🎮 DEMO OPTIONS:")
            for key, (name, _) in demo_options.items():
                print(f"{key}. {name}")

            choice = input("\nSelect option: ").strip()

            if choice == '6':
                print("👋 Demo completed!")
                break
            elif choice in demo_options:
                _, action = demo_options[choice]
                if action:
                    action()
            else:
                print("❌ Invalid option. Try again.")

    def show_system_status(self):
        """Show comprehensive system status"""
        print(f"\n🚀 SYSTEM STATUS OVERVIEW")
        print("=" * 30)

        if not self.is_initialized:
            print("❌ System not initialized")
            return

        # Component status
        print(f"📊 Component Status:")
        print(f"├─ AI System: {'✅ Active' if self.ai_system else '❌ Not loaded'}")
        print(f"├─ Database: {'✅ Connected' if self.database else '❌ Not connected'}")
        print(f"├─ Employee Manager: {'✅ Ready' if self.employee_manager else '❌ Not ready'}")
        print(f"├─ Video Processor: {'✅ Ready' if self.video_processor else '❌ Not ready'}")
        print(f"└─ Analytics: {'✅ Ready' if self.analytics_dashboard else '❌ Not ready'}")

        # Quick statistics
        if self.database:
            stats = self.database.get_statistics()
            print(f"\n📈 Quick Statistics:")
            print(f"├─ Employees: {stats.get('total_active_employees', 0)}")
            print(f"├─ Attendance Records: {stats.get('total_attendance_logs', 0)}")
            print(f"├─ Today's Activity: {stats.get('todays_attendance_count', 0)}")
            print(f"└─ Face Registrations: {stats.get('total_face_registrations', 0)}")

        # Performance overview
        if self.ai_system:
            ai_stats = self.ai_system.get_performance_stats()
            total_latency = ai_stats.get('avg_cpu_latency_ms', 0) + ai_stats.get('avg_gpu_latency_ms', 0)
            print(f"\n⚡ Performance Overview:")
            print(f"├─ Total Inferences: {ai_stats.get('total_inferences', 0)}")
            print(f"├─ Dual Stream: {'✅ Enabled' if ai_stats.get('dual_stream_enabled') else '❌ Disabled'}")
            print(f"└─ Avg Latency: {total_latency:.1f}ms")

    def demo_employee_management(self):
        """Demo employee management features"""
        print(f"\n👥 EMPLOYEE MANAGEMENT DEMO")
        print("=" * 35)

        if not self.employee_manager:
            print("❌ Employee manager not initialized")
            return

        # Show current employees
        self.employee_manager.show_registered_employees()

        if COLAB_ENV:
            print(f"\n📤 To register new employees:")
            print(f"employee_manager.upload_employee_photos()")
        else:
            print(f"\n💡 Employee management features:")
            print(f"├─ Upload photos: employee_manager.upload_employee_photos()")
            print(f"├─ Show employees: employee_manager.show_registered_employees()")
            print(f"└─ Get statistics: employee_manager.get_employee_statistics()")

    def demo_video_processing(self):
        """Demo video processing capabilities"""
        print(f"\n🎥 VIDEO PROCESSING DEMO")
        print("=" * 30)

        if not self.video_processor:
            print("❌ Video processor not initialized")
            return

        print(f"🎬 Video Processing Features:")
        print(f"├─ Dual-Stream Processing: CPU detection + GPU recognition")
        print(f"├─ Business Logic: 30min cooldown, work hours validation")
        print(f"├─ Real-time Analytics: Frame-by-frame tracking")
        print(f"└─ Attendance Recording: Automatic event logging")

        # Show processing stats if available
        if hasattr(self.video_processor, 'processing_stats'):
            stats = self.video_processor.processing_stats
            print(f"\n📊 Current Processing Stats:")
            print(f"├─ Frames Processed: {stats.get('frames_processed', 0)}")
            print(f"├─ Persons Detected: {stats.get('persons_detected', 0)}")
            print(f"├─ Faces Recognized: {stats.get('faces_recognized', 0)}")
            print(f"└─ Attendance Recorded: {stats.get('attendance_recorded', 0)}")

        if COLAB_ENV:
            print(f"\n📤 To process videos:")
            print(f"video_processor.upload_and_process_video()")

    def demo_analytics(self):
        """Demo analytics dashboard features"""
        print(f"\n📊 ANALYTICS DASHBOARD DEMO")
        print("=" * 35)

        if not self.analytics_dashboard:
            print("❌ Analytics dashboard not initialized")
            return

        # Show system dashboard
        self.analytics_dashboard.show_system_dashboard()

        # Show attendance report
        print(f"\n📋 Attendance Report (last 7 days):")
        self.analytics_dashboard.show_attendance_report(days_back=7)

        if COLAB_ENV:
            print(f"\n🎛️ Interactive controls available:")
            print(f"analytics_dashboard.create_interactive_widgets()")

    def demo_export(self):
        """Demo export functionality"""
        print(f"\n📤 EXPORT FUNCTIONALITY DEMO")
        print("=" * 35)

        if not self.analytics_dashboard:
            print("❌ Analytics dashboard not initialized")
            return

        # Export comprehensive reports
        self.analytics_dashboard.export_comprehensive_reports()

    def test_system_integration(self) -> bool:
        """Test complete system integration"""
        print(f"\n🧪 SYSTEM INTEGRATION TEST")
        print("=" * 30)

        if not self.is_initialized:
            print("❌ System not initialized")
            return False

        test_results = {
            'ai_system': False,
            'database': False,
            'employee_manager': False,
            'video_processor': False,
            'analytics': False
        }

        try:
            # Test AI system
            print(f"🤖 Testing AI system...")
            test_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
            ai_result = self.ai_system.process_dual_stream(test_image, test_image)
            test_results['ai_system'] = isinstance(ai_result, dict)
            print(f"   {'✅ PASS' if test_results['ai_system'] else '❌ FAIL'}")

            # Test database
            print(f"🗄️ Testing database...")
            db_stats = self.database.get_statistics()
            test_results['database'] = isinstance(db_stats, dict)
            print(f"   {'✅ PASS' if test_results['database'] else '❌ FAIL'}")

            # Test employee manager
            print(f"👥 Testing employee manager...")
            emp_stats = self.employee_manager.get_employee_statistics()
            test_results['employee_manager'] = isinstance(emp_stats, dict)
            print(f"   {'✅ PASS' if test_results['employee_manager'] else '❌ FAIL'}")

            # Test video processor
            print(f"🎥 Testing video processor...")
            proc_summary = self.video_processor.get_processing_summary()
            test_results['video_processor'] = isinstance(proc_summary, dict)
            print(f"   {'✅ PASS' if test_results['video_processor'] else '❌ FAIL'}")

            # Test analytics
            print(f"📊 Testing analytics...")
            health_score = self.analytics_dashboard._calculate_system_health_score()
            test_results['analytics'] = isinstance(health_score, (int, float))
            print(f"   {'✅ PASS' if test_results['analytics'] else '❌ FAIL'}")

        except Exception as e:
            logger.error(f"Integration test error: {e}")

        # Test summary
        passed_tests = sum(test_results.values())
        total_tests = len(test_results)

        print(f"\n📋 TEST SUMMARY:")
        print(f"├─ Passed: {passed_tests}/{total_tests}")
        print(f"├─ Success Rate: {passed_tests/total_tests*100:.1f}%")
        print(f"└─ Status: {'✅ ALL SYSTEMS OPERATIONAL' if passed_tests == total_tests else '⚠️ SOME ISSUES DETECTED'}")

        return passed_tests == total_tests

    def cleanup_system(self):
        """Cleanup system resources"""
        print(f"\n🧹 CLEANING UP SYSTEM RESOURCES")
        print("-" * 35)

        try:
            if self.ai_system:
                self.ai_system.cleanup()
                print("✅ AI system cleaned up")

            if self.database:
                self.database.close()
                print("✅ Database connection closed")

            print("✅ System cleanup completed")

        except Exception as e:
            logger.error(f"Cleanup error: {e}")

# Global convenience functions for easy usage
def init(model_pack='buffalo_l', demo_mode=True) -> PipelineV1System:
    """Initialize complete Pipeline V1 system"""
    system = PipelineV1System()
    success = system.initialize_system(model_pack=model_pack, demo_mode=demo_mode)

    if success:
        print(f"\n🎉 SYSTEM READY!")
        print(f"💡 Next steps:")
        print(f"   ├─ demo() - Run interactive demo")
        print(f"   ├─ upload_employees() - Register employees")
        print(f"   ├─ process_video() - Process attendance videos")
        print(f"   ├─ dashboard() - View analytics dashboard")
        print(f"   └─ export() - Export reports")

        # Store system globally for convenience functions
        globals()['_pipeline_system'] = system
        return system
    else:
        print(f"❌ System initialization failed")
        return None

def demo():
    """Run interactive demo"""
    if '_pipeline_system' in globals():
        globals()['_pipeline_system'].run_interactive_demo()
    else:
        print("❌ System not initialized. Run init() first.")

def upload_employees():
    """Upload employee photos"""
    if '_pipeline_system' in globals():
        system = globals()['_pipeline_system']
        if system.employee_manager:
            system.employee_manager.upload_employee_photos()
        else:
            print("❌ Employee manager not available")
    else:
        print("❌ System not initialized. Run init() first.")

def process_video():
    """Process attendance video"""
    if '_pipeline_system' in globals():
        system = globals()['_pipeline_system']
        if system.video_processor:
            system.video_processor.upload_and_process_video()
        else:
            print("❌ Video processor not available")
    else:
        print("❌ System not initialized. Run init() first.")

def dashboard():
    """Show analytics dashboard"""
    if '_pipeline_system' in globals():
        system = globals()['_pipeline_system']
        if system.analytics_dashboard:
            system.analytics_dashboard.show_system_dashboard()
        else:
            print("❌ Analytics dashboard not available")
    else:
        print("❌ System not initialized. Run init() first.")

def export():
    """Export comprehensive reports"""
    if '_pipeline_system' in globals():
        system = globals()['_pipeline_system']
        if system.analytics_dashboard:
            system.analytics_dashboard.export_comprehensive_reports()
        else:
            print("❌ Analytics dashboard not available")
    else:
        print("❌ System not initialized. Run init() first.")

def status():
    """Show system status"""
    if '_pipeline_system' in globals():
        globals()['_pipeline_system'].show_system_status()
    else:
        print("❌ System not initialized. Run init() first.")

def test():
    """Test system integration"""
    if '_pipeline_system' in globals():
        return globals()['_pipeline_system'].test_system_integration()
    else:
        print("❌ System not initialized. Run init() first.")
        return False

def cleanup():
    """Cleanup system resources"""
    if '_pipeline_system' in globals():
        globals()['_pipeline_system'].cleanup_system()
        del globals()['_pipeline_system']
    else:
        print("❌ System not initialized.")

# Auto-run welcome message
def show_welcome():
    """Show welcome message and quick start guide"""
    print(f"\n🎯 AI ATTENDANCE SYSTEM PIPELINE V1 - READY!")
    print("=" * 50)
    print(f"🚀 Quick Start Guide:")
    print(f"")
    print(f"1️⃣ Initialize System:")
    print(f"   system = init()  # or init('buffalo_s') for faster model")
    print(f"")
    print(f"2️⃣ Run Interactive Demo:")
    print(f"   demo()  # Interactive interface with all features")
    print(f"")
    print(f"3️⃣ Or Use Individual Functions:")
    print(f"   upload_employees()  # Register employee photos")
    print(f"   process_video()     # Process attendance videos")
    print(f"   dashboard()         # View analytics")
    print(f"   export()            # Export reports")
    print(f"   status()            # System status")
    print(f"   test()              # Test integration")
    print(f"   cleanup()           # Clean resources")
    print(f"")
    print(f"🎮 For full demo experience: init() → demo()")
    print("=" * 50)

# Initialize when imported
if __name__ == "__main__":
    show_welcome()
    print("✅ Cell 7 completed. Main execution system ready.")
    print("💡 Run init() to start the AI Attendance System!")
else:
    show_welcome()


🎯 AI ATTENDANCE SYSTEM PIPELINE V1 - READY!
🚀 Quick Start Guide:

1️⃣ Initialize System:
   system = init()  # or init('buffalo_s') for faster model

2️⃣ Run Interactive Demo:
   demo()  # Interactive interface with all features

3️⃣ Or Use Individual Functions:
   upload_employees()  # Register employee photos
   process_video()     # Process attendance videos
   dashboard()         # View analytics
   export()            # Export reports
   status()            # System status
   test()              # Test integration
   cleanup()           # Clean resources

🎮 For full demo experience: init() → demo()
✅ Cell 7 completed. Main execution system ready.
💡 Run init() to start the AI Attendance System!


# **Quick start**

In [8]:
# Copy-paste tất cả 7 cells vào Colab, sau đó:

# 1. Initialize complete system
system = init()

# 2. Run interactive demo
demo()

# 3. Or use individual functions
upload_employees()    # Upload employee photos
process_video()       # Process attendance videos
dashboard()           # View analytics
export()             # Export reports

ERROR:__main__:❌ System initialization failed: name 'setup_system' is not defined



🚀 CELL 7: AI ATTENDANCE SYSTEM PIPELINE V1
🎯 Complete system integration and demo capabilities
├─ Dual-Stream AI Processing
├─ SQLite Database with Vector Search
├─ Employee Management & Video Processing
├─ Analytics Dashboard & Reporting
└─ Interactive Demo & Testing

🔄 INITIALIZING PIPELINE V1 SYSTEM
├─ Model Pack: buffalo_l
├─ Demo Mode: True
└─ Environment: Google Colab

📦 Step 1: System Setup...
💡 Make sure all previous cells (1-6) are executed
❌ System initialization failed
❌ System not initialized. Run init() first.
❌ System not initialized. Run init() first.
❌ System not initialized. Run init() first.
❌ System not initialized. Run init() first.
❌ System not initialized. Run init() first.


# **Advanced Usage**

In [9]:
# Custom initialization
system = init(model_pack='buffalo_s', demo_mode=False)

# Direct component access
system.employee_manager.upload_employee_photos()
system.video_processor.upload_and_process_video()
system.analytics_dashboard.show_system_dashboard()

# System testing
test()  # Run integration tests
status()  # Show system status
cleanup()  # Clean resources

ERROR:__main__:❌ System initialization failed: name 'setup_system' is not defined



🚀 CELL 7: AI ATTENDANCE SYSTEM PIPELINE V1
🎯 Complete system integration and demo capabilities
├─ Dual-Stream AI Processing
├─ SQLite Database with Vector Search
├─ Employee Management & Video Processing
├─ Analytics Dashboard & Reporting
└─ Interactive Demo & Testing

🔄 INITIALIZING PIPELINE V1 SYSTEM
├─ Model Pack: buffalo_s
├─ Demo Mode: False
└─ Environment: Google Colab

📦 Step 1: System Setup...
💡 Make sure all previous cells (1-6) are executed
❌ System initialization failed


AttributeError: 'NoneType' object has no attribute 'employee_manager'